In [1]:
# %%
# 🧠 Local alias→label lookup using trained E5 model (no FAISS, offline)

import os
import json
import time
import numpy as np
import pandas as pd
import tempfile
from pathlib import Path
from sentence_transformers import SentenceTransformer, util

# ---------------------------------------------------------------------
# 🌐 Offline environment setup
# ---------------------------------------------------------------------
os.environ.update({
    "HF_HUB_OFFLINE": "1",
    "HF_DATASETS_OFFLINE": "1",
    "TRANSFORMERS_OFFLINE": "1",
    "HF_HUB_DISABLE_PROGRESS_BARS": "1",
    "TOKENIZERS_PARALLELISM": "false"
})

# ---------------------------------------------------------------------
# ⚙️ Auto-detect project root and paths
# ---------------------------------------------------------------------
if "__file__" in globals():
    PROJECT_ROOT = Path(__file__).resolve().parent.parent
else:
    PROJECT_ROOT = Path.cwd().resolve()
    if PROJECT_ROOT.name == "notebooks":
        PROJECT_ROOT = PROJECT_ROOT.parent  # step up to project root

MODEL_DIR = PROJECT_ROOT / "models" / "trained" / "alias_label_e5"
DATA_PATH = PROJECT_ROOT / "data" / "raw" / "dbpedia_schools.parquet"

TMP_DIR = Path(tempfile.gettempdir())
EMB_PATH = TMP_DIR / "edu_alias_label_embs.npy"
LABELS_PATH = TMP_DIR / "edu_alias_labels.json"

print(f"📁 Project root → {PROJECT_ROOT}")
print(f"🧠 Model path → {MODEL_DIR}")
print(f"📘 Dataset path → {DATA_PATH}")
print(f"💾 Cache dir → {TMP_DIR}\n")

📁 Project root → /home/craigtrim/projects/alias-label-retriever
🧠 Model path → /home/craigtrim/projects/alias-label-retriever/models/trained/alias_label_e5
📘 Dataset path → /home/craigtrim/projects/alias-label-retriever/data/raw/dbpedia_schools.parquet
💾 Cache dir → /tmp



In [2]:
# %%
# 🕒 Stopwatch helper
def timer(label: str = ""):
    start = time.perf_counter()
    def stop():
        elapsed = time.perf_counter() - start
        print(f"⏱️  {label} completed in {elapsed:,.2f}s ({elapsed/60:.2f} min)")
    return stop


# %%
# 🚀 Load local model
t = timer("Model load")
print(f"🚀 Loading local model from → {MODEL_DIR}")
model = SentenceTransformer(str(MODEL_DIR))
t()
print("✅ Model ready for inference\n")


# %%
# 📚 Load data
t = timer("Data load")
df = pd.read_parquet(DATA_PATH)
aliases = df["alias"].dropna().unique().tolist()
labels  = df["label"].dropna().unique().tolist()
t()
print(f"📘 Loaded {len(aliases):,} aliases and {len(labels):,} labels\n")


# %%
# 💾 Generate or load embeddings
if EMB_PATH.exists() and LABELS_PATH.exists():
    print(f"📦 Loading cached embeddings → {EMB_PATH}")
    label_embs = np.load(EMB_PATH)
    with open(LABELS_PATH) as f:
        labels = json.load(f)
else:
    t = timer("Embedding generation")
    print(f"🧩 Encoding {len(labels):,} labels (batch_size=32)…")
    label_embs = model.encode(
        labels,
        normalize_embeddings=True,
        show_progress_bar=False,   # disable progress bars entirely
        batch_size=32,
    )
    np.save(EMB_PATH, label_embs)
    with open(LABELS_PATH, "w") as f:
        json.dump(labels, f)
    t()
    print("✅ Embeddings generated and cached\n")

print(f"✅ Embeddings ready for lookup ({len(labels):,} labels)\n")

🚀 Loading local model from → /home/craigtrim/projects/alias-label-retriever/models/trained/alias_label_e5
⏱️  Model load completed in 8.09s (0.13 min)
✅ Model ready for inference

⏱️  Data load completed in 0.03s (0.00 min)
📘 Loaded 20,202 aliases and 20,272 labels

📦 Loading cached embeddings → /tmp/edu_alias_label_embs.npy
✅ Embeddings ready for lookup (20,272 labels)



In [10]:
# %%
def lookup(query: str, top_k: int = 10, min_score: float = 0.60):
    """Find most semantically similar university names for a given query."""
    # print(f"\n🔎 Query → {query}")
    q_emb = model.encode([query], normalize_embeddings=True, show_progress_bar=False)
    scores = util.cos_sim(q_emb, label_embs)[0]
    top = sorted(zip(labels, scores), key=lambda x: x[1], reverse=True)[:top_k]
    # print(f"📈 Top {top_k} matches:")
    results: list[dict] = []
    for rank, (lbl, score) in enumerate(top, 1):
        if score < min_score:
            continue
        results.append({
            "rank": rank, "label": lbl, "score": round(float(score), 3)
        })
        if round(float(score), 3) == 1.0:
            break # don't need more evidence 
        # print(f"  {rank:>2}. {lbl:<60} ({float(score):.4f})")
    # print("🏁 Lookup complete.\n")
    return results

# 🧪 Try a test lookup
print (lookup("YOUNG HARRIS COLLEGE", min_score = 0.5, top_k = 50))

[{'rank': 1, 'label': 'Houston Christian University', 'score': 0.567}, {'rank': 2, 'label': 'Campbell University', 'score': 0.563}, {'rank': 3, 'label': 'Magnolia Bible College', 'score': 0.558}, {'rank': 4, 'label': 'Wesley Junior College', 'score': 0.55}, {'rank': 5, 'label': 'Johnson C. Smith University', 'score': 0.549}, {'rank': 6, 'label': 'Hodges University', 'score': 0.549}, {'rank': 7, 'label': 'Lone Star College–North Harris', 'score': 0.545}, {'rank': 8, 'label': 'Cotton College', 'score': 0.541}, {'rank': 9, 'label': 'Southern Methodist College', 'score': 0.539}, {'rank': 10, 'label': 'Heritage Christian College', 'score': 0.531}, {'rank': 11, 'label': 'Hardin–Simmons University', 'score': 0.526}, {'rank': 12, 'label': 'Jacksonville College', 'score': 0.526}, {'rank': 13, 'label': 'Coleman University', 'score': 0.525}, {'rank': 14, 'label': 'Johnson State College', 'score': 0.525}, {'rank': 15, 'label': 'Cumberland University', 'score': 0.524}, {'rank': 16, 'label': 'Tennes

In [4]:
schools: list[str] = ['(USE #0261271) STLCC-MERAMEC', 'ABILENE CHRISTIAN UNIVERSITY', 'ABRAHAM BALDWIN AGRICULTURAL COLLEGE', 'ACADEMIA DEL SAGRADO CORAZON', 'ACADEMY ART UNIVERSITY', 'ACCOU-151', 'ACCRA TECHNICAL UNIVERSITY', 'ADAMS STATE UNIVERSITY', 'ADELPHI UNIVERSITY', 'ADRIAN COLLEGE', 'ADVENTIST UNIVERSITY HEALTH SCIENCES', 'AGNES SCOTT COLLEGE', 'AIB COLLEGE BUSINESS', 'AIKEN TECHNICAL COLLEGE', 'AIMS COMMUNITY COLLEGE', 'AIR FORCE INSTITUTE TECHNOLOGY', 'ALABAMA AGRICULTURAL MECHANICAL UNIVERSITY', 'ALABAMA STATE UNIVERSITY', 'ALAMANCE COMMUNITY COLLEGE', 'ALASKA PACIFIC UNIVERSITY', 'ALBANY STATE UNIVERSITY', 'ALBANY TECHNICAL COLLEGE', 'ALBERTUS MAGNUS COLLEGE', 'ALBIZU UNIVERSITY', 'ALBRIGHT COLLEGE', 'ALCORN STATE UNIVERSITY', 'ALDERSON BROADDUS C', 'ALEXANDRIA TECHNICAL COMMUNITY COLLEGE', 'ALFRED E BEACH HIGH SCHOOL', 'ALFRED STATE COLLEGE (SUNY)', 'ALFRED UNIVERSITY', 'ALICE LLOYD COLLEGE', 'ALLAN HANCOCK COLLEGE', 'ALLEGANY COLLEGE MARYLAND', 'ALLEGHENY COLLEGE', 'ALLEN COLLEGE', 'ALLEN COMMUNITY COLLEGE', 'ALLEN UNIVERSITY', 'ALLIANCE UNIVERSITY', 'ALLIANT INTL UNIV-SAN DIEGO', 'ALMA COLL', 'ALPENA COMMUNITY COLLEGE', 'ALTAMAHA TECHNICAL COLLEGE', 'ALTIERUS CAREER COLLEGE-TAMPA', 'ALVERNIA UNIVERSITY', 'ALVERNO COLLEGE', 'ALVIN COMMUNITY COLLEGE', 'AMARILLO COLLEGE', 'AMBASSADOR COLLEGE (CA)', 'AMBERTON UNIVERSITY', 'AMERICAN ACADEMY DRAMATIC ARTS', 'AMERICAN COUNCIL ON EDUCATION', 'AMERICAN INTERCONTINENTAL UNIVERSITY (PRIMARY)', 'AMERICAN INTERNATIONAL COLLEGE', 'AMERICAN PUBLIC UNIVERSITY SYSTEM', 'AMERICAN PUBLIC UNIVERSITY SYSTEM (PRIMARY)', 'AMERICAN RIVER COLLEGE', 'AMERICAN SAMOA COMMUNITY COLLEGE', 'AMERICAN UNIVERSITY', 'AMERICAN UNIVERSITY PARIS', 'AMERICAN UNIVERSITY PUERTO RICO', 'AMERICAN UNIVERSITY ROME', 'AMERITECH COLLEGE HEALTHCARE', 'AMHERST COLLEGE', 'ANA G. MéNDEZ UNIVERSITY', 'ANCILLA COLLEGE', 'ANDERSON UNIVERSITY (IN)', 'ANDERSON UNIVERSITY (SC)', 'ANDREW COLLEGE', 'ANDREWS UNIVERSITY', 'ANGELINA COLL', 'ANGELO STATE UNIVERSITY', 'ANGLO-AMERICKA VYSOKA SKOLA', 'ANNA MARIA COLLEGE', 'ANNE ARUNDEL COMMUNITY COLLEGE', 'ANOKA RAMSEY COMMUNITY COLLEGE', 'ANOKA TECHNICAL COLLEGE', 'ANTELOPE VALLEY COLLEGE', 'ANTIOCH UNIVERSITY', 'ANTIOCH UNIVERSITY - LOS ANGELES', 'APPALACHIAN STATE UNIVERSITY', 'AQUINAS COLLEGE (MI)', 'AQUINAS COLLEGE (TN)', 'ARAPAHOE COMMUNITY COLLEGE', 'ARCADIA UNIVERSITY', 'ARGOSY UNIVERSITY (PRIMARY)', 'ARIZONA CHRISTIAN UNIVERSITY', 'ARIZONA STATE UNIVERSITY (PRIMARY)', 'ARIZONA WESTERN COLL', 'ARKANSAS BAPTIST COLLEGE', 'ARKANSAS NORTHEASTERN COLLEGE', 'ARKANSAS ST UNIV MOUNTAIN HOME', 'ARKANSAS STATE UNIVERSITY', 'ARKANSAS STATE UNIVERSITY - BEEBE', 'ARKANSAS STATE UNIVERSITY - NEWPORT', 'ARKANSAS STATE UNIVERSITY MID-SOUTH', 'ARKANSAS STATE UNIVERSITY THREE RIVERS', 'ARKANSAS TECH UNIVERSITY', 'ARMSTRONG STATE UNIVERSITY', 'ART ACADEMY CINCINNATI', 'ART INSTITUTE ATLANTA', 'ART INSTITUTE AUSTIN', 'ART INSTITUTE CALIFORNIA-HOLLYWOOD', 'ART INSTITUTE CALIFORNIA-SAN DIEGO', 'ART INSTITUTE DALLAS', 'ART INSTITUTE FT. LAUDERDALE', 'ART INSTITUTE HOUSTON', 'ART INSTITUTE LAS VEGAS', 'ART INSTITUTE PHILADELPHIA', 'ART INSTITUTE PITTSBURGH', 'ART INSTITUTE SEATTLE', 'ART INSTITUTE TENNESSEE-NASHVILLE', 'ART INSTITUTE VIRGINIA BEACH', 'ART INSTITUTE WASHINGTON', 'ASA COLLEGE', 'ASBURY UNIVERSITY', 'ASHEVILLE-BUNCOMBE TECHNICAL COMMUNITY COLLEGE', 'ASHLAND UNIVERSITY', 'ASNUNTUCK COMMUNITY COLLEGE', 'ASSINIBOINE COMMUNITY COLLEGE', 'ATC-240', 'ATCHISON HS', 'ATHABASCA UNIVERSITY', 'ATHENS STATE UNIVERSITY', 'ATHENS TECHNICAL COLLEGE', 'ATLANTA METROPOLITAN STATE COLLEGE', 'ATLANTA TECHNICAL COLLEGE', 'ATLANTIC CAPE COMMUNITY COLLEGE', 'AUBURN UNIVERSITY', 'AUBURN UNIVERSITY MONTGOMERY', 'AUGSBURG UNIVERSITY', 'AUGUSTA TECHNICAL COLLEGE', 'AUGUSTA UNIVERSITY', 'AUGUSTANA COLLEGE', 'AUGUSTANA UNIVERSITY', 'AURORA UNIVERSITY', 'AUSTIN COLL', 'AUSTIN COMMUNITY COLLEGE DISTRICT', 'AUSTIN PEAY STATE UNIVERSITY (PRIMARY)', 'AVE MARIA UNIVERSITY', 'AVERETT UNIVERSITY', 'AVILA UNIVERSITY', 'AZUSA PACIFIC UNIVERSITY', 'BACONE COLLEGE', 'BAINBRIDGE STATE COLLEGE', 'BAKER COLLEGE (MI) - PRIMARY', 'BAKER UNIVERSITY', 'BAKERSFIELD COLLEGE', 'BALDWIN WALLACE UNIVERSITY', 'BALL STATE UNIVERSITY', 'BALTIMORE CITY COMMUNITY COLLEGE', 'BAPTIST BIBLE COLLGE', 'BAPTIST COLLEGE FLORIDA,', 'BAPTIST COLLEGE HEALTH SCIENCES', 'BARCLAY COLLEGE (KS)', 'BARD COLLEGE', 'BARRY UNIVERSITY', 'BARSTOW COMMUNITY COLLEGE', 'BARTON COLLEGE', 'BARTON COUNTY COMMUNITY COLLEGE', 'BARUCH COLLEGE CAMPUS HS', 'BASTYR UNIVERSITY', 'BATES COLLEGE', 'BATES TECHNICAL COLLEGE', 'BATON ROUGE COMMUNITY COLLEGE', 'BAUDER COLLEGE', 'BAY DE NOC COMMUNITY COLLEGE', 'BAY MILLS COMMUNITY COLLEGE', 'BAY PATH UNIVERSITY', 'BAY STATE COLLEGE', 'BAYLOR UNIVERSITY', 'BECK AREA VOCATIONAL CENTER', 'BECKER COLLEGE', 'BELHAVEN UNIVERSITY', 'BELLARMINE UNIVERSITY', 'BELLEVUE COLLEGE', 'BELLEVUE UNIVERSITY', 'BELLIN COLLEGE', 'BELLINGHAM TECHNICAL COLLEGE', 'BELMONT ABBEY COLLEGE', 'BELMONT COLLEGE', 'BELMONT UNIVERSITY', 'BELOIT COLLEGE', 'BEMIDJI STATE UNIVERSITY', 'BENEDICT COLLEGE', 'BENEDICTINE COLLEGE', 'BENEDICTINE UNIVERSITY', 'BENJAMIN FRANKLIN INSTITUTE TECHNOLOGY', 'BENNETT COLLEGE', 'BENTLEY UNIVERSITY', 'BEREA COLLEGE', 'BERGEN COMMUNITY COLLEGE', 'BERKELEY CITY COLLEGE', 'BERKELEY COLL-WEST PATERSON', 'BERKELEY COLLEGE (NY)', 'BERKELEY COLLEGE - WOODBRIDGE', 'BERKELEY COLLEGE-WESTCHESTER', 'BERKLEE COLLEGE MUSIC', 'BERKSHIRE COMMUNITY COLLEGE', 'BERMUDA COLLEGE', 'BERNARD M BARUCH COLLEGE', 'BERRY COLLEGE', 'BETHANY COLLEGE (KS)', 'BETHANY LUTHERAN COLLEGE', 'BETHANY UNIVERSITY', 'BETHEL UNIVERSITY (MN)', 'BETHEL UNIVERSITY (TN)', 'BETHUNE-COOKMAN UNIVERSITY', 'BEVILL STATE COMMUNITY COLLEGE (PRIMARY)', 'BIG BEND COMMUNITY COLLEGE', 'BINGHAMTON UNIVERSITY SUNY', 'BIO-113', 'BIOLA UNIVERSITY', 'BIRMINGHAM-SOUTHERN COLLEGE', 'BISHOP STATE COMMUNITY COLLEGE', 'BISMARCK STATE COLLEGE', 'BLACK HAWK COLLEGE (PRIMARY)', 'BLACK HILLS STATE UNIVERSITY', 'BLACK RIVER TECHNICAL COLLEGE', 'BLACKBURN COLLEGE', 'BLACKHAWK TECHNICAL COLLEGE', 'BLADEN COMMUNITY COLLEGE', 'BLINN COLLEGE', 'BLOOMFIELD COLLEGE', 'BLOOMSBURG UNIVERSITY PENNSYLVANIA', 'BLUE MOUNTAIN COLLEGE', 'BLUE MOUNTAIN COMMUNITY COLLEGE', 'BLUE RIDGE COMM TECH COLL', 'BLUE RIDGE COMMUNITY COLLEGE', 'BLUE RIDGE COMMUNITY COLLEGE (VA)', 'BLUEFIELD COLLEGE (VA)', 'BLUEFIELD STATE COLLEGE (WV) - PRIMARY', 'BLUFFTON UNIVERSITY', 'BOB JONES UNIVERSITY', 'BOISE STATE UNIVERSITY', 'BOROUGH MANHATTAN COMMUNITY COLLEGE', 'BOSSIER PARISH COMMUNITY COLLEGE', 'BOSTON COLLEGE', 'BOSTON UNIVERSITY', 'BOW VALLEY COLLEGE', 'BOWDOIN COLLEGE', 'BOWIE STATE UNIVERSITY', 'BOWLING GREEN STATE UNIVERSITY (PRIMARY)', 'BRADLEY UNIVERSITY', 'BRAZOSPORT COLLEGE', 'BRENAU UNIVERSITY', 'BRESCIA UNIVERSITY', 'BREWTON-PARKER COLLEGE', 'BRIAR CLIFF UNIVERSITY', 'BRIARCLIFFE COLLEGE (PRIMARY)', 'BRIDGEVALLEY COMMUNITY TECHNICAL COLLLEGE (PRIMARY)', 'BRIDGEWATER COLLEGE', 'BRIDGEWATER STATE UNIVERSITY', 'BRIGHAM YOUNG U HI', 'BRIGHAM YOUNG UNIV. -IDAHO', 'BRIGHAM YOUNG UNIVERSITY', 'BRIGHTPOINT COMMUNITY COLLEGE', 'BRIGHTWOOD COLLEGE', 'BRISTOL COMMUNITY COLLEGE', 'BRITISH COLUMBIA INSTITUTE TECHNOLOGY', 'BRONX COMMUNITY COLLEGE', 'BROOKDALE COMMUNITY COLLEGE', 'BROOKLYN COLLEGE (CUNY)', 'BROOKS COLLEGE', 'BROOME COMMUNITY COLLEGE', 'BROWARD COLLEGE (PRIMARY)', 'BROWN MACKIE COLLEGE (PRIMARY)', 'BRUNSWICK COMMUNITY COLLEGE', 'BRYAN COLL HEALTH SCIENCES', 'BRYAN COLLEGE', 'BRYANT STRATTON COLLEGE (PRIMARY)', 'BRYN ATHYN COLLEGE', 'BRYN MAWR COLLEGE', 'BUCKNELL UNIV', 'BUCKS COUNTY COMMUNITY COLLEGE', 'BUDP-0116', 'BUENA VISTA UNIVERSITY', 'BUFFALO STATE COLLEGE', 'BUNKER HILL COMMUNITY COLLEGE', 'BURLINGTON COLLEGE', 'BUSN-141', 'BUTLER COMMUNITY COLLEGE', 'BUTLER COUNTY COMMUNITY COLLEGE (PA)', 'BUTLER UNIVERSITY', 'BUTTE COLLEGE', 'CABRILLO COLLEGE', 'CABRINI COLLEGE', 'CAIRN UNIVERSITY', 'CALDWELL COMMUNITY COLLEGE TECHNICAL INSTITUTE', 'CALDWELL UNIVERSITY', 'CALHOUN COMMUNITY COLLEGE', 'CALIF C ARTS CRAFTS', 'CALIF INSTITUTE ARTS', 'CALIFORNIA BAPTIST UNIVERSITY', 'CALIFORNIA LUTHERAN UNIVERSITY', 'CALIFORNIA POLYTECHNIC STATE UNIVERSITY', 'CALIFORNIA STATE POLYTECHNIC UNIVERSITY POMONA', 'CALIFORNIA STATE UNIVERSITY BAKERSFIELD', 'CALIFORNIA STATE UNIVERSITY CHANNEL ISLANDS', 'CALIFORNIA STATE UNIVERSITY CHICO', 'CALIFORNIA STATE UNIVERSITY DOMINGUEZ HILLS', 'CALIFORNIA STATE UNIVERSITY EAST BAY', 'CALIFORNIA STATE UNIVERSITY FRESNO', 'CALIFORNIA STATE UNIVERSITY FULLERTON', 'CALIFORNIA STATE UNIVERSITY LONG BEACH', 'CALIFORNIA STATE UNIVERSITY LOS ANGELES', 'CALIFORNIA STATE UNIVERSITY MONTEREY BAY', 'CALIFORNIA STATE UNIVERSITY NORTHRIDGE', 'CALIFORNIA STATE UNIVERSITY SACRAMENTO', 'CALIFORNIA STATE UNIVERSITY SAN BERNARDINO', 'CALIFORNIA STATE UNIVERSITY SAN MARCOS', 'CALIFORNIA STATE UNIVERSITY STANISLAUS', 'CALVIN UNIVERSITY', 'CAMDEN COUNTY COLLEGE', 'CAMERON UNIVERSITY (PRIMARY)', 'CAMPBELL UNIVERSITY', 'CAMPBELLSVILLE UNIVERSITY', 'CANADA COLL', 'CANISIUS COLLEGE', 'CAPE COD COMMUNITY COLLEGE', 'CAPE FEAR COMMUNITY COLLEGE', 'CAPELLA UNIVERSITY', 'CAPITAL COMMUNITY COLLEGE', 'CAPITAL UNIVERSITY', 'CARDINAL STRITCH UNIVERSITY', 'CARL ALBERT STATE COLLEGE (PRIMARY)', 'CARL SANDBURG COLLEGE', 'CARLOW UNIVERSITY', 'CARNEGIE-MELLON UNIV', 'CARRINGTON COLLEGE', 'CARROLL COLLEGE', 'CARROLL COMMUNITY COLLEGE', 'CARROLL UNIVERSITY', 'CARSON-NEWMAN UNIVERSITY', 'CARTERET COMMUNITY COLLEGE', 'CARTHAGE COLLEGE', 'CASCADIA COLLEGE', 'CASE WESTERN RESERVE UNIVERSITY', 'CASPER COLLEGE', 'CASTLETON ST COLL', 'CATAWBA COLL', 'CATAWBA VALLEY COMMUNITY COLLEGE', 'CATHOLIC DISTANCE UNIV', 'CATHOLIC UNIVERSITY AMERICA', 'CAYUGA COMMUNITY COLLEGE', 'CAZENOVIA COLLEGE', 'CECIL COLLEGE', 'CEDAR CREST COLL', 'CEDARVILLE UNIVERSITY', 'CENTENARY COLLEGE LOUISIANA', 'CENTENARY UNIVERSITY', 'CENTRAL ALABAMA COMMUNITY COLLEGE', 'CENTRAL ARIZONA COLLEGE', 'CENTRAL BAPTIST COLLEGE', 'CENTRAL CAROLINA COMMUNITY COLLEGE', 'CENTRAL CAROLINA TECHNICAL COLLEGE', 'CENTRAL CHRISTIAN COLLEGE KANSAS', 'CENTRAL COLLEGE', 'CENTRAL COMMUNITY COLLEGE (NE) - PRIMARY', 'CENTRAL CONNECTICUT STATE UNIVERSITY', 'CENTRAL GEORGIA TECHNICAL COLLEGE', 'CENTRAL LAKES COLLEGE', 'CENTRAL MAINE COMMUNITY COLLEGE', 'CENTRAL METHODIST UNIVERSITY', 'CENTRAL MICHIGAN UNIVERSITY', 'CENTRAL NEW MEXICO COMMUNITY COLLEGE', 'CENTRAL OHIO TECHNICAL COLLEGE', 'CENTRAL OREGON COMMUNITY COLLEGE', 'CENTRAL PENNSYLVANIA COLLEGE', 'CENTRAL PIEDMONT COMMUNITY COLLEGE', 'CENTRAL STATE UNIVERSITY (OH)', 'CENTRAL TEXAS COLLEGE', 'CENTRAL VIRGINIA COMMUNITY COLLEGE', 'CENTRAL WASHINGTON UNIVERSITY', 'CENTRAL WYOMING COLLEGE', 'CENTRALIA COLLEGE', 'CENTRE COLLEGE', 'CENTURY COLLEGE', 'CERRITOS COLLEGE', 'CERRO COSO COMMUNITY COLLEGE', 'CHABOT COLL', 'CHADRON ST COLL', 'CHADRON STATE COLLEGE', 'CHAFFEY COLLEGE', 'CHAMBERLAIN UNIVERSITY (PRIMARY)', 'CHAMINADE UNIVERSITY HONOLULU', 'CHAMPLAIN COLL', 'CHARLES R DREW UNIVERSITY MEDICINE SCIENCE', 'CHARLESTON SOUTHERN UNIVERSITY', 'CHARTER COLLEGE', 'CHARTER OAK STATE COLLEGE', 'CHATHAM UNIVERSITY', 'CHATTAHOOCHEE TECHNICAL COLLEGE', 'CHATTAHOOCHEE VALLEY COMMUNITY COLLEGE', 'CHATTANOOGA STATE COMMUNITY COLLEGE', 'CHEM-1110', 'CHEMEKETA COMMUNITY COLLEGE', 'CHESAPEAKE COLL', 'CHESTNUT HILL COLL', 'CHEYNEY UNIV PENNSYLVANIA', 'CHICAGO CITY COLLEGE', 'CHICAGO SCHOOL PROFESSIONAL PSYCH', 'CHICAGO STATE UNIVERSITY', 'CHIPOLA COLLEGE', 'CHIPPEWA VALLEY TECHNICAL COLLEGE', 'CHOWAN COLLEGE', 'CHRISTENDOM COLLEGE', 'CHRISTIAN BROTHERS COLLEGE HIGH SCHOOL', 'CHRISTIAN BROTHERS UNIVERSITY', 'CHRISTOPHER NEWPORT UNIVERSITY', 'CINCINNATI STATE TECHNICAL COMMUNITY COLLEGE', 'CIS-125', 'CISCO COLLEGE', 'CITADEL', 'CITRUS COLLEGE', 'CITY COLLEGE AT MONTANA STATE UNIVERSITY BILLINGS', 'CITY COLLEGE CHICAGO', 'CITY COLLEGE NEW YORK', 'CITY COLLEGE SAN FRANCISCO', 'CITY COLLEGES CHICAGO (PRIMARY)', 'CITY UNIVERSITY SEATTLE', 'CLACKAMAS COMMUNITY COLLEGE', 'CLAFLIN COLLEGE', 'CLARENDON COLLEGE', 'CLARK ATLANTA UNIVERSITY', 'CLARK COLLEGE', 'CLARK STATE COMMUNITY COLLEGE', 'CLARK UNIVERSITY', 'CLARKE UNIVERSITY', 'CLARKSON COLLEGE', 'CLARKSON UNIV', 'CLAYTON STATE UNIVERSITY', 'CLEMSON UNIV', 'CLEP', 'CLEVELAND COMMUNITY COLLEGE', 'CLEVELAND STATE COMMUNITY COLLEGE', 'CLEVELAND STATE UNIVERSITY', 'CLEVELAND UNIVERSITY', 'CLINCH VALLEY COLL', 'CLINTON COMMUNITY COLLEGE (NY)', 'CLOUD COUNTY COMMUNITY COLLEGE', 'CLOVER PARK TECHNICAL COLLEGE', 'CLOVIS COMMUNITY COLLEGE (CA)', 'CLOVIS COMMUNITY COLLEGE (NM)', 'COAHOMA COMMUNITY COLLEGE', 'COAST COMMUNITY COLLEGE DISTRICT', 'COASTAL ALABAMA COMMUNITY COLLEGE', 'COASTAL BEND COLLEGE', 'COASTAL CAROLINA COMMUNITY COLLEGE', 'COASTAL CAROLINA UNIVERSITY', 'COASTLINE COMMUNITY COLLEGE', 'COCHISE COLLEGE', 'COCONINO COMMUNITY COLLEGE', 'COE COLLEGE', 'COFFEYVILLE COMMUNITY COLLEGE (PRIMARY)', 'COGSWELL POLYTECHNICAL COLLEGE', 'COKER COLL', 'COLBY COLL', 'COLBY COMMUNITY COLLEGE', 'COLBY-SAWYER COLLEGE', 'COLEGIO AMERICANO DE QUITO', 'COLL ALBEMARLE', 'COLL ST JOSEPH PROVD', 'COLL WILLIAM MARY', 'COLLEGE ALAMEDA', 'COLLEGE CANYONS', 'COLLEGE CENTRAL FLORIDA', 'COLLEGE CHARLESTON', 'COLLEGE COASTAL GEORGIA', 'COLLEGE DESERT', 'COLLEGE DUPAGE', 'COLLEGE EASTERN IDAHO', 'COLLEGE EASTERN UTAH', 'COLLEGE LAKE COUNTY', 'COLLEGE MAINLAND', 'COLLEGE MARIN', 'COLLEGE MOUNT SAINT VINCENT', 'COLLEGE NEW JERSEY', 'COLLEGE NEW ROCHELLE', 'COLLEGE OZARKS', 'COLLEGE REDWOODS', 'COLLEGE SAINT MARY', 'COLLEGE SAINT ROSE', 'COLLEGE SAN MATEO', 'COLLEGE SEQUOIAS', 'COLLEGE SISKIYOUS', 'COLLEGE SOUTHERN IDAHO', 'COLLEGE SOUTHERN MARYLAND', 'COLLEGE SOUTHERN NEVADA', 'COLLEGE ST. SCHOLASTICA', 'COLLEGE STATEN ISLAND', 'COLLEGE WESTCHESTER', 'COLLEGE WESTERN IDAHO', 'COLLEGE WOOSTER', 'COLLIN COUNTY COMMUNITY COLLEGE', 'COLORADO CHRISTIAN UNIVERSITY', 'COLORADO COLLEGE', 'COLORADO MESA UNIVERSITY', 'COLORADO MOUNTAIN COLLEGE (PRIMARY)', 'COLORADO NORTHWESTERN COMMUNITY COLLEGE', 'COLORADO SCHOOL MINES', 'COLORADO STATE UNIVERSITY - FT. COLLINS', 'COLORADO STATE UNIVERSITY - GLOBAL', 'COLORADO STATE UNIVERSITY - PUEBLO', 'COLORADO TECHNICAL UNIVERSITY (PRIMARY)', 'COLUMBIA BASIN COLLEGE', 'COLUMBIA CENTRAL UNIVERSITY', 'COLUMBIA COLLEGE', 'COLUMBIA COLLEGE (CA)', 'COLUMBIA COLLEGE (SC)', 'COLUMBIA COLLEGE IL', 'COLUMBIA COLLEGE MO', 'COLUMBIA STATE COMMUNITY COLLEGE', 'COLUMBIA UNIVERSITY IN CITY NEW YORK', 'COLUMBIA-GREENE COMMUNITY COLLEGE', 'COLUMBUS COLLEGE ART DESIGN', 'COLUMBUS STATE COMMUNITY COLLEGE', 'COLUMBUS STATE UNIVERSITY', 'COLUMBUS TECHNICAL COLLEGE', 'COMMUNITY COLLEGE AIR FORCE', 'COMMUNITY COLLEGE ALLEGHENY COUNTY', 'COMMUNITY COLLEGE AURORA', 'COMMUNITY COLLEGE BALTIMORE COUNTY (PRIMARY)', 'COMMUNITY COLLEGE BEAVER COUNTY', 'COMMUNITY COLLEGE DENVER', 'COMMUNITY COLLEGE PHILADELPHIA', 'COMMUNITY COLLEGE RHODE ISLAND', 'COMMUNITY COLLEGE VERMONT', 'COMPTON COLLEGE', 'CONCORD UNIVERSITY', 'CONCORDIA COLLEGE (AL)', 'CONCORDIA COLLEGE (MN)', 'CONCORDIA COLLEGE ANN ARBOR', 'CONCORDIA COLLEGE NY', 'CONCORDIA UNIVERSITY (CANADA)', 'CONCORDIA UNIVERSITY (NE)', 'CONCORDIA UNIVERSITY (OR)', 'CONCORDIA UNIVERSITY - IRVINE', 'CONCORDIA UNIVERSITY IL', 'CONCORDIA UNIVERSITY ST. PAUL', 'CONCORDIA UNIVERSITY TEXAS', 'CONCORDIA UNIVERSITY-WISCONSIN', 'CONNECTICUT COLL', 'CONNORS STATE COLLEGE (PRIMARY)', 'CONSERVATORY MUSIC PUERTO RICO', 'CONTRA COSTA COLLEGE', 'CONVERSE COLLEGE', 'COPIAH-LINCOLN COMMUNITY COLLEGE (PRIMARY)', 'COPPER MOUNTAIN COLLEGE', 'COPPIN STATE UNIVERSITY', 'COQUITLAM COLLEGE', 'CORNELL COLLEGE', 'CORNELL UNIV', 'CORNERSTONE UNIVERSITY', 'CORNING COMMUNITY COLLEGE', 'CORNISH COLLEGE ARTS', 'COSSATOT COMMUNITY COLLEGE UA', 'COSUMNES RIVER COLLEGE', 'COTTEY COLLEGE', 'COUNTY COLLEGE MORRIS', 'COVENANT COLL', 'COWLEY COUNTY COMMUNITY COLLEGE', 'COX COLLEGE', 'CRAFTON HILLS COLLEGE', 'CRAVEN COMMUNITY COLLEGE', 'CREIGHTON UNIVERSITY', 'CRISWELL COLLEGE', 'CROWDER COLLEGE', "CROWLEY'S RIDGE COLLEGE", 'CROWN COLLEGE', 'CSD-316', 'CUESTA COLLEGE', 'CULINARY INSTITUTE AMERICA', 'CULVER-STOCKTON COLLEGE', 'CUMBERLAND UNIVERSITY', 'CUNY GRADUATE SCHL UNIV CTR', 'CUNY GRADUATE SCHOOL UNIVERSITY CENTER', 'CURRY COLLEGE', 'CUYAHOGA COMMUNITY COLLEGE (PRIMARY)', 'CUYAMACA COLLEGE', 'CYPRESS COLLEGE', "D'YOUVILLE COLLEGE", 'DAEMEN COLLEGE', 'DAKOTA COLLEGE AT BOTTINEAU', 'DAKOTA COUNTY TECHNICAL COLLEGE', 'DAKOTA STATE COLLEGE', 'DAKOTA STATE UNIVERSITY', 'DAKOTA WESLEYAN UNIVERSITY', 'DALHOUSIE UNIV', 'DALLAS BAPTIST UNIVERSITY', 'DALLAS COLLEGE (PRIMARY)', 'DALTON STATE COLLEGE', 'DANA COLLEGE', 'DANIEL WEBSTER COLL', 'DANVILLE AREA COMMUNITY COLLEGE', 'DANVILLE COMMUNITY COLLEGE', 'DAVENPORT UNIVERSITY', 'DAVID LIPSCOMB UNIVERSITY', 'DAVIDSON COLLEGE', 'DAVIDSON COUNTY COMMUNITY COLLEGE', 'DAVIDSON-DAVIE COMM COLL', 'DAVIS ELKINS COLLEGE', 'DAWSON COMMUNITY COLLEGE', 'DAYTONA STATE COLLEGE', 'DE ANZA COLLEGE', 'DEAN COLLEGE', 'DEFENSE LANGUAGE INSTITUTE', 'DEFIANCE COLLEGE', 'DEL MAR COLLEGE', 'DELAWARE COUNTY COMMUNITY COLLEGE', 'DELAWARE STATE UNIVERSITY', 'DELAWARE TECHNICAL COMMUNITY COLLEGE (PRIMARY)', 'DELAWARE VALLEY COLLEGE', 'DELGADO COMMUNITY COLLEGE (PRIMARY)', 'DELTA COLLEGE', 'DELTA STATE UNIVERSITY', 'DENISON JOB CORPS', 'DENISON UNIVERSITY', 'DENMARK TECHNICAL COLLEGE', 'DENVER SCHOOL NURSING', 'DEPAUL UNIVERSITY', 'DEPAUW UNIVERSITY', 'DEPT DEFENSE', 'DES MOINES AREA COMMUNITY COLLEGE (PRIMARY)', 'DESALES UNIVERSITY', 'DEVRY UNIVERSITY - PRIMARY', 'DIABLO VALLEY COLLEGE', 'DICKINSON COLLEGE', 'DICKINSON STATE UNIVERSITY', 'DILLARD UNIVERSITY', 'DINE COLLEGE', 'DO NOT USE', 'DOANE UNIVERSITY', 'DODGE CITY COMMUNITY COLLEGE', 'DOMINICAN COLLEGE (NY)', 'DOMINICAN UNIVERSITY (IL)', 'DOMINICAN UNIVERSITY CALIFORNIA', 'DONNELLY COLLEGE', 'DOWLING COLLEGE', 'DRAKE UNIVERSITY', 'DREW UNIVERSITY', 'DREXEL UNIVERSITY', 'DRURY UNIVERSITY', 'DUBLIN HIGH SCHOOL', 'DUKE UNIVERSITY', 'DUNWOODY COLLEGE TECHNOLOGY', 'DUQUESNE UNIVERSITY', 'DURBAN UNIVERSITY TECHNOLOGY', 'DURHAM TECHNICAL COMMUNITY COLLEGE', 'DUTCHESS COMMUNITY COLLEGE', 'DYERSBURG STATE COMMUNITY COLLEGE', 'EARLHAM COLLEGE', 'EAST ARKANSAS COMMUNITY COLLEGE', 'EAST CAROLINA UNIVERSITY', 'EAST CENTRAL COLLEGE', 'EAST CENTRAL COMMUNITY COLLEGE (MS)', 'EAST CENTRAL UNIVERSITY', 'EAST COAST POLYTECHNIC INSTITUTE UNIVERSITY', 'EAST GEORGIA STATE COLLEGE', 'EAST LOS ANGELES COLLEGE', 'EAST MISSISSIPPI COMMUNITY COLLEGE (PRIMARY)', 'EAST STROUDSBURG UNIVERSITY PENNSYLVANIA', 'EAST TENNESSEE STATE UNIVERSITY', 'EAST TEXAS BAPTIST UNIVERSITY', 'EAST-WEST UNIVERSITY', 'EASTERN ARIZONA COLLEGE', 'EASTERN CONNECTICUT STATE UNIVERSITY', 'EASTERN FLORIDA STATE COLLEGE', 'EASTERN GATEWAY COMMUNITY COLLEGE', 'EASTERN ILLINOIS UNIVERSITY', 'EASTERN INTERNATIONAL COLLEGE', 'EASTERN IOWA COMMUNITY COLLEGE DISTRICT', 'EASTERN IOWA COMMUNITY COLLEGES (PRIMARY)', 'EASTERN KENTUCKY UNIVERSITY (PRIMARY)', 'EASTERN MAINE COMMUNITY COLLEGE', 'EASTERN MENNONITE UNIVERSITY', 'EASTERN MICHIGAN UNIVERSITY', 'EASTERN NAZARENE COLLEGE', 'EASTERN NEW MEXICO UNIVERSITY', 'EASTERN NEW MEXICO UNIVERSITY - ROSWELL (CC)', 'EASTERN OKLAHOMA STATE COLLEGE (PRIMARY)', 'EASTERN OREGON UNIVERSITY', 'EASTERN SHORE COMMUNITY COLLEGE', 'EASTERN UNIVERSITY (PA)', 'EASTERN WASHINGTON UNIVERSITY', 'EASTERN WYOMING COLLEGE', 'ECKERD COLLEGE', 'EDGECOMBE COMMUNITY COLLEGE', 'EDGEWOOD COLLEGE', 'EDISON COMMUNITY COLLEGE', 'EDMONDS COMMUNITY COLLEGE', 'EDWARD W CLARK HIGH SCHOOL', 'EDWARD WATERS COLLEGE', 'EL CAMINO COLLEGE', 'EL PASO COMMUNITY COLLEGE', 'ELGIN COMMUNITY COLLEGE', 'ELIZABETH CITY STATE UNIVERSITY', 'ELIZABETHTOWN COLLEGE', 'ELLSWORTH COMMUNITY COLLEGE', 'ELMHURST COLLEGE', 'ELMIRA COLLEGE', 'ELMS COLLEGE', 'ELON UNIVERSITY', 'EMBRY-RIDDLE AERO UNIV WORLDWIDE', 'EMBRY-RIDDLE AERONAUTICAL UNIVERSITY', 'EMBRY-RIDDLE AERONAUTICAL UNIVERSITY - PRESCOTT', 'EMERSON COLLEGE', 'EMMANUEL COLLEGE (GA)', 'EMMAUS BIBLE COLLEGE', 'EMORY HENRY COLLEGE', 'EMORY UNIVERSITY', 'EMPORIA STATE UNIVERSITY', 'ENDICOTT COLLEGE', 'ENG-110', 'ENSIGN COLLEGE', 'ENTERPRISE STATE COMMUNITY COLLEGE', 'ERASMUS UNIVERSITY ROTTERDAM', 'ERIE COMMUNITY COLLEGE (PRIMARY)', 'ESSEX COUNTY COLLEGE', 'EUREKA COLLEGE', 'EVANGEL UNIVERSITY', 'EVEREST COLLEGE', 'EVERETT COMMUNITY COLLEGE', 'EVERGREEN STATE COLLEGE', 'EVERGREEN VALLEY COLLEGE', 'EXCELSIOR ACADEMY', 'EXCELSIOR COLLEGE', 'FAIRFIELD UNIVERSITY', 'FAIRLEIGH DICKINSON UNIVERSITY (PRIMARY)', 'FAIRMONT STATE UNIVERSITY', 'FASHION INSTITUTE TECHNOLOGY', 'FASHION INSTOF DESIGN MERCH', 'FAULKNER UNIVERSITY', 'FAULKNER UNIVERSITY-MOBILE', 'FAYETTEVILLE STATE UNIVERSITY', 'FAYETTEVILLE TECHNICAL COMMUNITY COLLEGE', 'FEATHER RIVER COLLEGE', 'FELICIAN UNIVERSITY', 'FERRIS STATE UNIVERSITY', 'FERRUM COLLEGE', 'FINGER LAKES COMMUNITY COLLEGE', 'FINLANDIA UNIVERSITY', 'FISHER COLLEGE', 'FISK UNIVERSITY', 'FITCHBURG STATE UNIVERSITY', 'FIVE TOWNS COLLEGE', 'FLAGLER COLLEGE', 'FLATHEAD VALLEY COMMUNITY COLLEGE', 'FLINT HILLS TECHNICAL COLL', 'FLINT HILLS TECHNICAL COLLEGE', 'FLORENCE-DARLINGTON TECHNICAL COLLEGE', 'FLORIDA A&M UNIVERSITY', 'FLORIDA ATLANTIC UNIVERSITY', 'FLORIDA COLLEGE', 'FLORIDA GATEWAY COLLEGE', 'FLORIDA GULF COAST UNIVERSITY', 'FLORIDA INSTITUTE TECHNOLOGY', 'FLORIDA INTERNATIONAL UNIVERSITY', 'FLORIDA KEYS COMMUNITY COLLEGE', 'FLORIDA MEMORIAL UNIVERSITY', 'FLORIDA NATIONAL UNIVERSITY', 'FLORIDA SOUTHERN COLLEGE', 'FLORIDA SOUTHWESTERN STATE COLLEGE', 'FLORIDA STATE COLLEGE AT JACKSONVILLE', 'FLORIDA STATE UNIVERSITY', 'FLORIDA STATE UNIVERSITY SCHOOL DISTRICT', 'FOLSOM LAKE COLLEGE', 'FOND DU LAC TRIBAL COMMUNITY COLLEGE', 'FOOTHILL COLLEGE', 'FORDHAM UNIVERSITY', 'FORSYTH TECHNICAL COMMUNITY COLLEGE', 'FORT HAYS STATE UNIVERSITY', 'FORT LEWIS COLLEGE', 'FORT SCOTT COMMUNITY COLLEGE', 'FORT VALLEY STATE UNIVERSITY', 'FORTIS COLLEGE', 'FOX COLLEGE', 'FOX VALLEY TECHNICAL COLLEGE', 'FRAMINGHAM STATE COLLEGE', 'FRANCIS MARION UNIVERSITY', 'FRANCISCAN MISSIONARIES OUR LADY UNIVERSITY', 'FRANCISCAN UNIVERSITY STEUBENVILLE', 'FRANK PHILLIPS COLLEGE', 'FRANKLIN COLLEGE INDIANA', 'FRANKLIN PIERCE COLLEGE', 'FRANKLIN UNIVERSITY', 'FRANKLIN UNIVERSITY SWITZERLAND', 'FREDERICK COMMUNITY COLLEGE', 'FREDERICK DOUGLASS HIGH SCHOOL', 'FREED-HARDEMAN UNIVERSITY (PRIMARY)', 'FRESNO CITY COLLEGE', 'FRESNO PACIFIC UNIVERSITY', 'FRIENDS UNIVERSITY', 'FRONT RANGE COMMUNITY COLLEGE (PRIMARY)', 'FRONTIER NURSING UNIVERSITY', 'FROSTBURG STATE UNIVERSITY', 'FULLERTON COLLEGE', 'FULTON-MONTGOMERY COMMUNITY COLLEGE', 'FURMAN UNIVERSITY', 'GADSDEN STATE COMMUNITY COLLEGE (PRIMARY)', 'GALEN COLLEGE NURSING (PRIMARY)', 'GALEN SCHOOL NURSING', 'GALLAUDET UNIVERSITY', 'GALVESTON COLLEGE', 'GANNON UNIVERSITY', 'GARDEN CITY COMMUNITY COLLEGE', 'GARDNER-WEBB UNIVERSITY', 'GARRETT COLLEGE', 'GASTON COLLEGE', 'GATEWAY COMMUNITY COLLEGE', 'GATEWAY TECHNICAL COLLEGE', 'GAVILAN COLLEGE', 'GBA-319', 'GENESEE COMMUNITY COLLEGE', 'GENEVA COLLEGE', 'GEORGE BROWN COLLEGE', 'GEORGE FOX UNIVERSITY', 'GEORGE MASON UNIVERSITY', 'GEORGE WASHINGTON UNIVERSITY', 'GEORGETOWN COLL', 'GEORGETOWN UNIVERSITY', 'GEORGIA COLLEGE STATE UNIVERSITY', 'GEORGIA GWINNETT COLLEGE', 'GEORGIA HIGHLANDS COLLEGE', 'GEORGIA INSTITUTE TECHNOLOGY', 'GEORGIA MILITARY COLLEGE', 'GEORGIA MILITARY COLLEGE HS', 'GEORGIA NORTHWESTERN TECHNICAL COLLEGE', 'GEORGIA PERIMETER COLLEGE', 'GEORGIA PIEDMONT TECHNICAL COLLEGE', 'GEORGIA SOUTHERN UNIVERSITY', 'GEORGIA SOUTHWESTERN STATE UNIVERSITY', 'GEORGIA STATE UNIVERSITY', 'GEORGIAN COLLEGE', 'GEORGIAN COURT UNIVERSITY', 'GERMANNA COMMUNITY COLLEGE', 'GETTYSBURG COLLEGE', 'GLEN OAKS COMMUNITY COLLEGE', 'GLENDALE COMMUNITY COLLEGE (CA)', 'GLENVILLE STATE COLLEGE', 'GODDARD COLLEGE', 'GOGEBIC COMMUNITY COLLEGE', 'GOLDEN GATE UNIVERSITY', 'GOLDEN WEST COLLEGE', 'GOLDEY-BEACOM COLLEGE', 'GOLDFARB SCHOOL NURSING AT BARNES-JEWISH COLLEGE', 'GONZAGA UNIVERSITY', 'GOOD SAMARITAN COLLEGE NURSING HEALTH SCIENCE', 'GOODWIN UNIVERSITY', 'GORDON COLLEGE', 'GORDON STATE COLLEGE', 'GOSHEN COLLEGE', 'GOUCHER COLLEGE', 'GOVERNORS STATE UNIVERSITY', 'GRACE CHRISTIAN UNIVERSITY', 'GRACE COLLEGE SEMINARY', 'GRACELAND UNIVERSITY', 'GRAMBLING STATE UNIVERSITY', 'GRAND CANYON UNIVERSITY', 'GRAND RAPIDS COMMUNITY COLLEGE', 'GRAND VALLEY STATE UNIVERSITY', 'GRAND VIEW UNIVERSITY', 'GRANITE STATE COLLEGE', 'GRANTHAM UNIVERSITY', 'GRAYS HARBOR COLLEGE', 'GRAYSON COLLEGE', 'GREAT BASIN COLLEGE', 'GREAT BAY COMMUNITY COLLEGE', 'GREAT LAKES CHRISTIAN COLLEGE', 'GREEN RIVER COLLEGE', 'GREENFIELD COMMUNITY COLLEGE', 'GREENVILLE TECHNICAL COLLEGE', 'GREENVILLE UNIVERSITY', 'GRINNELL COLLEGE', 'GROSSMONT COLLEGE', 'GROVE CITY COLLEGE', 'GUAM COMMUNITY COLLEGE', 'GUILFORD COLLEGE', 'GUILFORD TECHNICAL COMMUNITY COLLEGE', 'GULF COAST STATE COLLEGE', 'GULFPORT', 'GUTTMAN COMMUNITY COLLEGE', 'GWINNETT TECHNICAL COLLEGE', 'GWYNEDD MERCY UNIVERSITY', 'HAGERSTOWN COMMUNITY COLLEGE', 'HALIFAX COMMUNITY COLLEGE', 'HAMILTON HS EAST - STEINERT', 'HAMLINE UNIVERSITY', 'HAMPTON UNIVERSITY', 'HANNIBAL-LAGRANGE UNIVERSITY', 'HANOVER COLLEGE', 'HARCUM COLLEGE', 'HARDIN-SIMMONS UNIVERSITY', 'HARDING UNIVERSITY', 'HARFORD COMMUNITY COLLEGE', 'HARPER COLLEGE', 'HARRIS-STOWE STATE UNIVERSITY', 'HARRISBURG AREA COMMUNITY COLLEGE', 'HARRISBURG UNIVERSITY SCIENCE TECHNOLOGY', 'HARTNELL COLLEGE', 'HARTWICK COLLEGE', 'HARVARD UNIVERSITY', 'HASKELL INDIAN NATIONS UNIVERSITY', 'HASTINGS COLL', 'HAWAII COMMUNITY COLLEGE', 'HAWAII PACIFIC UNIVERSITY', 'HAWKEYE COMMUNITY COLLEGE', 'HAYWOOD COMMUNITY COLLEGE', 'HEALD COLLEGE', 'HEALTHCARE MANAGEMENT PROFESSIONAL', 'HEALTHCARE MANAGEMENT PROFESSIONAL LICENSE', 'HEARTLAND COMMUNITY COLLEGE', 'HELENA COLLEGE UNIVERSITY MONTANA', 'HELENE FULD COLLEGE NURSING', 'HELLENIC COLLEGE HOLY CROSS', 'HENDERSON STATE UNIVERSITY', 'HENDRIX COLLEGE', 'HENNEPIN TECHNICAL COLLEGE (PRIMARY)', 'HENRY FORD COLLEGE', 'HERKIMER COUNTY COMMUNITY COLLEGE', 'HERZING C-MINNEAPOLIS DRAFTING SCHOOL', 'HERZING UNIVERSITY', 'HERZING UNIVERSITY - HOMEWOOD', 'HESSTON COLLEGE', 'HIBBING COMMUNITY COLLEGE', 'HIGH POINT UNIVERSITY', 'HIGHLAND COMMUNITY COLLEGE (IL)', 'HIGHLAND COMMUNITY COLLEGE (KS)', 'HIGHLAND COMMUNITY SCHOOL', 'HIGHLAND PARK COMMUNITY COLLEGE', 'HIGHLINE COLLEGE', 'HILBERT COLLEGE', 'HILL COLLEGE', 'HILLSBOROUGH COMMUNITY COLLEGE', 'HINDS COMMUNITY COLLEGE (PRIMARY)', 'HIRAM COLLEGE', 'HIS-201', 'HIST-213', 'HOCKING COLLEGE', 'HODGES UNIVERSITY', 'HOFSTRA UNIVERSITY', 'HOLLINS UNIVERSITY', 'HOLMES COMMUNITY COLLEGE (PRIMARY)', 'HOLY FAMILY UNIVERSITY', 'HOLY NAMES UNIVERSITY', 'HOLYOKE COMMUNITY COLLEGE', 'HONDROS COLLEGE', 'HONOLULU COMMUNITY COLLEGE', 'HOOD COLLEGE', 'HOPE COLLEGE', 'HOPE INTERNATIONAL UNIVERSITY', 'HORRY-GEORGETOWN TECHNICAL COLLEGE', 'HOSTOS COMMUNITY COLLEGE', 'HOUGHTON COLL HGHTON', 'HOUSATONIC COMMUNITY COLLEGE', 'HOUSTON BAPTIST UNIVERSITY', 'HOUSTON COMMUNITY COLLEGE', 'HOWARD COLL BIG SPRG', 'HOWARD COMMUNITY COLLEGE', 'HOWARD PAYNE UNIVERSITY', 'HOWARD UNIVERSITY', 'HUDSON COUNTY COMMUNITY COLLEGE', 'HUDSON VALLEY COMMUNITY COLLEGE', 'HUMBOLDT STATE UNIVERSITY', 'HUMPHREYS COLL', 'HUNTER COLLEGE (CUNY)', 'HUNTINGDON COLLEGE', 'HUNTINGTON UNIVERSITY', 'HUSSON UNIVERSITY', 'HUSTON-TILLOTSON UNIVERSITY', 'HUTCHINSON COMMUNITY COLLEGE', 'IDAHO STATE UNIVERSITY', 'ILLINOIS CENTRAL COLLEGE', 'ILLINOIS COLLEGE', 'ILLINOIS EASTERN COMMUNITY COLLEGES (PRIMARY)', 'ILLINOIS INST ART AT SCHAUMBURG', 'ILLINOIS INSTITUTE ART', 'ILLINOIS INSTITUTE TECHNOLOGY', 'ILLINOIS STATE UNIVERSITY', 'ILLINOIS VALLEY COMMUNITY COLLEGE', 'ILLINOIS WESLEYLAN UNIVERSITY', "ILOILO DOCTORS' COLLEGE", 'IMMACULATA UNIVERSITY', 'IMPERIAL VALLEY COLLEGE', 'INDEPENDENCE COMMUNITY COLLEGE', 'INDIAN HILLS COMMUNITY COLLEGE (PRIMARY)', 'INDIAN RIVER STATE COLLEGE', 'INDIANA INSTITUTE TECHNOLOGY', 'INDIANA STATE UNIVERSITY', 'INDIANA UNIV FT WYNE', 'INDIANA UNIV KOKOMO', 'INDIANA UNIV NORTHWEST', 'INDIANA UNIV-PURDUE UNIV AT COLUMBUS', 'INDIANA UNIVERSITY - BLOOMINGTON', 'INDIANA UNIVERSITY EAST', 'INDIANA UNIVERSITY PENNSYLVANIA', 'INDIANA UNIVERSITY PURDUE UNIVERSITY INDIANAPOLIS', 'INDIANA UNIVERSITY SOUTH BEND', 'INDIANA UNIVERSITY SOUTHEAST', 'INDIANA WESLEYAN UNIVERSITY', 'INTER AMERICAN UNIVERSITY PUERTO RICO', 'INTERBORO INSTITUTE', 'INTERNATIONAL COLLEGE', 'INTERNATIONAL COLLEGE OR UNIVERSITY 1', 'INVER HILLS COMMUNITY COLLEGE', 'IONA UNIVERSITY', 'IOWA CENTRAL COMMUNITY COLLEGE (PRIMARY)', 'IOWA LAKES COMMUNITY COLLEGE (PRIMARY)', 'IOWA STATE UNIVERSITY', 'IOWA WESLEYAN COLLEGE', 'IOWA WESTERN COMMUNITY COLLEGE', 'IRVINE VALLEY COLLEGE', 'ISOTHERMAL COMMUNITY COLLEGE', 'ISTANBUL TECHNICAL UNIVERSITY', 'ITASCA COMMUNITY COLLEGE', 'ITAWAMBA COMMUNITY COLLEGE (PRIMARY)', 'ITHACA COLL', 'IVY TECH COMMUNITY COLLEGE (PRIMARY)', 'J H FAULKNER STATE COMMUNITY COLLEGE', 'J. F. DRAKE STATE TECHNICAL COLLEGE', 'J. SARGEANT REYNOLDS COMMUNITY COLLEGE', 'JACKSON COLLEGE', 'JACKSON STATE COMMUNITY COLLEGE', 'JACKSON STATE UNIVERSITY', 'JACKSONVILLE COLLEGE', 'JACKSONVILLE STATE UNIVERSITY', 'JACKSONVILLE UNIVERSITY', 'JAMES MADISON UNIVERSITY', 'JAMES SPRUNT COMMUNITY COLLEGE', 'JAMESTOWN BUSINESS COLLEGE', 'JAMESTOWN COMMUNITY COLLEGE', 'JARVIS CHRISTIAN COLLEGE', 'JAWAHARLAL NEHRU TECHNOLOGICAL UNIVERSITY HYDERABAD', 'JEFFERSON COLLEGE', 'JEFFERSON COLLEGE HEALTH SCIENCES', 'JEFFERSON COMMUNITY COLLEGE (NY)', 'JEFFERSON STATE COMMUNITY COLLEGE (PRIMARY)', 'JERSEY COLLEGE', 'JEWISH HOSPITAL', 'JOHN A GUPTON COLLEGE', 'JOHN A LOGAN COLLEGE', 'JOHN BROWN UNIVERSITY', 'JOHN CARROLL UNIVERSITY', 'JOHN JAY COLLEGE CRIMINAL JUSTICE', 'JOHN MARSHALL LAW SCHOOL', 'JOHN PAUL GREAT CATHOLIC UNIVERSITY', 'JOHN WOOD COMMUNITY COLLEGE', 'JOHNS HOPKINS UNIVERSITY', 'JOHNSON C SMITH UNIVERSITY', 'JOHNSON COUNTY COMMUNITY COLLEGE', 'JOHNSON STATE COLLEGE', 'JOHNSON UNIVERSITY', 'JOHNSON WALES UNIVERSITY (PRIMARY)', 'JOHNSTON COMMUNITY COLLEGE', 'JOLIET JUNIOR COLLEGE', 'JONES COUNTY JUNIOR COLLEGE (PRIMARY)', 'JONES INTERNATIONAL UNIVERSITY', 'JOUR-150', 'JUDSON COLLEGE (AL)', 'JUDSON UNIVERSITY', 'JUNIATA COLLEGE', 'KALAMAZOO VALLEY COMMUNITY COLLEGE', 'KANKAKEE COMMUNITY COLLEGE', 'KANSAS CITY ART INSTITUTE', 'KANSAS CITY KANSAS COMMUNITY COLLEGE', 'KANSAS STATE UNIVERSITY', 'KANSAS WESLEYAN UNIVERSITY', "KAPI'OLANI COMMUNITY COLLEGE", 'KASKASKIA COLLEGE', "KAUA'I COMMUNITY COLLEGE", 'KEAN UNIVERSITY', 'KEENE STATE COLLEGE', 'KEISER UNIVERSITY (PRIMARY)', 'KELLOGG COMMUNITY COLLEGE', 'KEMPER MILITARY SCHOOL', 'KENDALL COLLEGE', 'KENNEBEC VALLEY COMMUNITY COLLEGE', 'KENNESAW STATE UNIVERSITY', 'KENRICK-GLENNON SEMINARY', 'KENT STATE UNIV-SALEM CAMPUS', 'KENT STATE UNIVERSITY (PRIMARY)', 'KENTUCKY CHRISTIAN UNIVERSITY', 'KENTUCKY COMMUNITY TECHNICAL COLLEGE SYSTEM (PRIMARY)', 'KENTUCKY STATE UNIVERSITY', 'KENTUCKY WESLEYAN COLLEGE', 'KENYATTA UNIVERSITY', 'KENYON COLLEGE', 'KETTERING UNIVERSITY', 'KEUKA COLLEGE', 'KEYSTONE COLLEGE', 'KILGORE COLLEGE', 'KING SAUD UNIVERSITY', 'KING UNIVERSITY', "KING'S COLLEGE NY", 'KINGS COLL PA', 'KINGSBOROUGH COMMUNITY COLLEGE (CUNY)', 'KIRKWOOD COMMUNITY COLLEGE', 'KIRTLAND COMMUNITY COLLEGE', 'KISHWAUKEE COLLEGE', 'KLAMATH COMMUNITY COLLEGE', 'KNOX COLLEGE', 'KPED-1122', 'KUTZTOWN UNIVERSITY PENNSYLVANIA', 'KWAME NKRUMAH UNIVERSITY SCIENCE TECHNOLOGY', 'LA ROCHE COLLEGE', 'LA SALLE UNIVERSITY', 'LA SIERRA UNIVERSITY', 'LA TROBE UNIVERSITY', 'LABETTE COMMUNITY COLLEGE', 'LACKAWANNA COLLEGE', 'LAFAYETTE COLLEGE', 'LAGRANGE COLLEGE', 'LAGUARDIA COMMUNITY COLLEGE', 'LAKE AREA TECHNICAL INSTITUTE', 'LAKE ERIE COLLEGE', 'LAKE ERIE COLLEGE OSTEOPATHIC MEDICINE', 'LAKE FOREST COLLEGE', 'LAKE LAND COLLEGE', 'LAKE MICHIGAN COLLEGE', 'LAKE REGION STATE COLLEGE', 'LAKE SUPERIOR COLLEGE', 'LAKE SUPERIOR STATE UNIVERSITY', 'LAKE TAHOE COMMUNITY COLLEGE', 'LAKE WASHINGTON INSTITUTE TECHNOLOGY', 'LAKE-SUMTER STATE COLLEGE', 'LAKELAND COMMUNITY COLLEGE', 'LAKELAND UNIVERSITY', 'LAKES REGION COMMUNITY COLLEGE', 'LAKESHORE TECHNICAL COLLEGE', 'LAKEVIEW COLLEGE NURSING', 'LAMAR COMMUNITY COLLEGE', 'LAMAR INSTITUTE TECHNOLOGY', 'LAMAR STATE COLLEGE', 'LAMAR UNIV', 'LAMBUTH COLL', 'LANCASTER BIBLE COLL', 'LANDER UNIVERSITY', 'LANDMARK COLLEGE', 'LANE COLLEGE', 'LANE COMMUNITY COLLEGE', 'LANEY COLLEGE', 'LANGSTON UNIVERSITY', 'LANIER TECHNICAL COLLEGE', 'LANKENAU HOSPITAL', 'LANSING COMMUNITY COLLEGE', 'LARAMIE COUNTY COMMUNITY COLLEGE', 'LAREDO COMMUNITY COLLEGE', 'LAS POSITAS COLLEGE', 'LASELL COLLEGE', 'LASSEN COMMUNITY COLLEGE', 'LAUREL RIDGE COMMUNITY COLLEGE', 'LAURENTIAN UNIVERSITY', 'LAWRENCE TECHNOLOGICAL UNIVERSITY', 'LAWRENCE UNIVERSITY', 'LAWSON STATE COMMUNITY COLLEGE', 'LE CORDON BLEU COLLEGE CULINARY ARTS (PRIMARY)', 'LE CORDON BLEU COLLEGE CULINARY ARTS - CHICAGO', 'LE MOYNE COLLEGE', 'LEBANON VALLEY COLLEGE', 'LEE COLLEGE', 'LEE UNIVERSITY', 'LEES-MCRAE COLLEGE', 'LEEWARD COMMUNITY COLLEGE', 'LEHIGH CARBON COMMUNITY COLLEGE', 'LEHMAN COLLEGE', 'LEMOYNE-OWEN COLLEGE', 'LENOIR COMMUNITY COLLEGE', 'LENOIR-RHYNE UNIVERSITY', 'LETOURNEAU UNIVERSITY', 'LEWIS CLARK COLLEGE (OR)', 'LEWIS CLARK COMMUNITY COLLEGE', 'LEWIS UNIVERSITY', 'LEWIS-CLARK STATE COLLEGE', 'LEXINGTON COLLEGE', 'LIBERTY UNIVERSITY', 'LIFE PACIFIC COLLEGE', 'LIFE UNIVERSITY', 'LIMESTONE UNIVERSITY', 'LINCOLN CHRISTIAN UNIVERSITY', 'LINCOLN COLLEGE (PRIMARY)', 'LINCOLN COLLEGE NEW ENGLAND', 'LINCOLN LAND COMMUNITY COLLEGE', 'LINCOLN MEMORIAL UNIVERSITY', 'LINCOLN UNIVERSITY', 'LINCOLN UNIVERSITY LABORATORY HIGH SCHOOL', 'LINDSEY WILSON COLLEGE', 'LINN-BENTON COMMUNITY COLLEGE', 'LITTLE BIG HORN COLLEGE', 'LITTLE PRIEST TRIBAL COLLEGE', 'LIU POST', 'LIVINGSTONE COLLEGE', 'LLT-321', 'LOCK HAVEN UNIV PENNSYLVANIA', 'LOGAN UNIVERSITY', 'LOMA LINDA UNIVERSITY', 'LONE STAR COLLEGE (PRIMARY)', 'LONG BEACH CITY COLLEGE', 'LONG ISLAND UNIVERSITY BROOKLYN', 'LONGWOOD UNIVERSITY', 'LORAIN COUNTY COMMUNITY COLLEGE', 'LORAS COLLEGE', 'LOS ANGELES CITY COLLEGE', 'LOS ANGELES HARBOR COLLEGE', 'LOS ANGELES MISSION COLLEGE', 'LOS ANGELES PACIFIC UNIVERSITY', 'LOS ANGELES PIERCE COLLEGE', 'LOS ANGELES SOUTHWEST COLLEGE', 'LOS ANGELES TRADE TECHNICAL COLLEGE', 'LOS ANGELES VALLEY COLLEGE', 'LOS MEDANOS COLLEGE', 'LOUISBURG COLLEGE', 'LOUISIANA CHRISTIAN UNIVERSITY', 'LOUISIANA DELTA COMMUNITY COLLEGE (PRIMARY)', 'LOUISIANA STATE UNIVERSITY A&M COLLEGE', 'LOUISIANA STATE UNIVERSITY AT ALEXANDRIA', 'LOUISIANA STATE UNIVERSITY EUNICE', 'LOUISIANA STATE UNIVERSITY HEALTH SCIENCES CENTER', 'LOUISIANA STATE UNIVERSITY SHREVEPORT', 'LOUISIANA TECH UNIV', 'LOURDES UNIVERSITY', 'LOWER COLUMBIA COLL', 'LOYOLA MARYMNT UNIV', 'LOYOLA UNIVERSITY CHICAGO', 'LOYOLA UNIVERSITY MARYLAND', 'LOYOLA UNIVERSITY NEW ORLEANS', 'LUBBOCK CHRISTIAN UNIVERSITY', 'LUTHER COLLEGE (IA)', 'LUTHERAN MED SCH NURSING', 'LUZERNE COUNTY COMMUNITY COLLEGE', 'LYCOMING COLL', 'LYNDON STATE COLLEGE', 'LYNN UNIVERSITY', 'LYON COLLEGE', 'MACC ADV TECH CENTER', 'MACCORMAC COLLEGE', 'MACEWAN UNIVERSITY', 'MACMURRAY COLLEGE', 'MACOMB COMMUNITY COLLEGE', 'MACONAQUAH HIGH SCHOOL', 'MADERA COMMUNITY COLLEGE', 'MADISON AREA TECHNICAL COLLEGE', 'MADONNA UNIVERSITY', 'MAHARISHI INTERNATIONAL UNIVERSITY', 'MAINE COLLEGE HEALTH PROFESSIONS', 'MAKERERE UNIVERSITY', 'MALONE UNIVERSITY', 'MANCHESTER COMMUNITY COLLEGE (CT)', 'MANCHESTER COMMUNITY COLLEGE (NH)', 'MANCHESTER UNIVERSITY', 'MANHATTAN AREA TECHNICAL COLLEGE', 'MANHATTAN SCHOOL MUSIC', 'MANHATTANVILLE COLL', 'MANOR COLLEGE', 'MANSFIELD UNIVERSITY PENNSYLVANIA', 'MARIA COLLEGE ALBANY', 'MARIAN UNIVERSITY (IN)', 'MARIAN UNIVERSITY-WISCONSIN', 'MARICOPA COMMUNITY COLLEGE DISTRICT', 'MARICOPA COMMUNITY COLLEGES (PRIMARY)', 'MARIETTA COLL', 'MARION MILITARY INSTITUTE', 'MARION TECHNICAL COLLEGE', 'MARIST COLLEGE', 'MARQUETTE UNIVERSITY', 'MARS HILL COLL', 'MARSHALL UNIVERSITY', 'MARSHALLTOWN COMMUNITY COLLEGE', 'MARTIN COMMUNITY COLLEGE', 'MARTIN METHODIST COLLEGE', 'MARTIN UNIVERSITY', 'MARY BALDWIN UNIVERSITY', 'MARYCREST COLLEGE', 'MARYLAND COMM COLLEGE', 'MARYLAND INSTITUTE COLLEGE ART', 'MARYMOUNT CALIFORNIA UNIVERSITY', 'MARYMOUNT COLLEGE (KS)', 'MARYMOUNT MANHATTN C', 'MARYMOUNT UNIVERSITY (VA)', 'MARYVILLE COLLEGE (TN)', 'MARYVILLE UNIVERSITY-ST LOUIS', 'MARYWOOD COLLEGE', 'MASSACHUSETTS BAY COMMUNITY COLLEGE', 'MASSACHUSETTS COLLEGE LIBERAL ARTS', 'MASSACHUSETTS INSTITUTE TECHNOLOGY', 'MASSASOIT COMMUNITY COLLEGE', "MASTER'S COLLEGE,", 'MAUR HILL PREP SCHOOL', 'MAYVILLE STATE UNIVERSITY', 'MCDANIEL COLLEGE', 'MCDOWELL TECHNICAL COMMUNITY COLLEGE', 'MCHENRY COUNTY COLLEGE', 'MCKENDREE UNIVERSITY', 'MCLENNAN COMMUNITY COLLEGE', 'MCMURRY UNIVERSITY', 'MCNEESE STATE UNIVERSITY', 'MCPHERSON COLLEGE', 'MCPHS UNIVERSITY', 'MED-134', 'MEDAILLE COLLEGE', 'MEDGAR EVERS COLLEGE', 'MEDICAL UNIVERSITY SOUTH CAROLINA', 'MEDICINE HAT COLLEGE', 'MEMPHIS COLLEGE ART', 'MENDOCINO COLLEGE', 'MENLO COLLEGE', 'MERCED COLLEGE', 'MERCER COUNTY COMMUNITY COLLEGE', 'MERCER UNIV MACON', 'MERCY COLL NORTHWEST OHIO', 'MERCY COLLEGE', 'MERCY COLLEGE HEALTH SCIENCES', 'MERCYHURST UNIVERSITY', 'MERIDIAN JR COLL', 'MERRIMACK COLLEGE', 'MERRITT COLLEGE', 'MESABI RANGE COLLEGE (PRIMARY)', 'MESALANDS COMMUNITY COLLEGE', 'MESSIAH COLLEGE', 'METHODIST COLLEGE NURSING', 'METHODIST UNIVERSITY', 'METROPOLITAN COLLEGE NEW YORK', 'METROPOLITAN COMMUNITY COLLEGE', 'METROPOLITAN COMMUNITY COLLEGE (MO) - PRIMARY', 'METROPOLITAN COMMUNITY COLLEGE (NE)', 'METROPOLITAN STATE UNIVERSITY (MN)', 'METROPOLITAN STATE UNIVERSITY DENVER', 'MGH INSTITUTE HEALTH PROFESSIONS', 'MIAMI DADE COLLEGE', 'MIAMI UNIVERSITY', 'MIAMI UNIVERSITY (OH) - PRIMARY', 'MICHIGAN LUTHERAN COLLEGE', 'MICHIGAN STATE UNIVERSITY', 'MICHIGAN TECHNOLOGICAL UNIVERSITY', 'MID MICHIGAN COMMUNITY COLLEGE', 'MID PLAINS COMMUNITY COLLEGE', 'MID-AMERICA CHRISTIAN UNIVERSITY', 'MID-ATLANTIC CHRISTIAN UNIVERSITY', 'MID-CONTINENT UNIVERSITY', 'MID-PLAINS COMMUNITY COLLEGE', 'MID-STATE TECHNICAL COLLEGE', 'MIDAMERICA NAZARENE UNIVERSITY', 'MIDDLE COLLEGE AT BENNETT', 'MIDDLE GEORGIA STATE UNIVERSITY', 'MIDDLE TENNESSEE STATE UNIVERSITY', 'MIDDLEBURY COLLEGE', 'MIDDLESEX COMMUNITY COLLEGE', 'MIDDLESEX COMMUNITY COLLEGE (CT)', 'MIDDLESEX COUNTY COLLEGE', 'MIDLAND COLLEGE', 'MIDLAND UNIVERSITY', 'MIDLANDS TECHNICAL COLLEGE', 'MIDSTATE COLLEGE', 'MIDWAY UNIVERSITY (PRIMARY)', 'MIDWESTERN BAPTIST THEOLOGICAL SEMINARY', 'MIDWESTERN STATE UNIVERSITY', 'MILES COLLEGE', 'MILES COMMUNITY COLLEGE', 'MILLERSVILLE UNIVERSITY PENNSYLVANIA', 'MILLIGAN COLLEGE', 'MILLIKIN UNIVERSITY', 'MILLSAPS COLLEGE', 'MILWAUKEE AREA TECHNICAL COLLEGE', 'MILWAUKEE INSTOF ART DESIGN', 'MILWAUKEE SCH ENGR', 'MINERAL AREA COLLEGE', 'MINNEAPOLIS COMMUNITY TECHNICAL COLLEGE', 'MINNESOTA STATE COLLEGE', 'MINNESOTA STATE COLLEGE SOUTHEAST', 'MINNESOTA STATE COMMUNITY TECHNICAL COLLEGE', 'MINNESOTA STATE UNIVERSITY - MANKATO', 'MINNESOTA STATE UNIVERSITY MOORHEAD', 'MINNESOTA WEST C TECH C-GRANITE FALLS', 'MINNESOTA WEST COMMUNITY TECHNICAL COLLEGE', 'MINOT STATE UNIVERSITY', 'MIRACOSTA COLLEGE', 'MISERICORDIA UNIVERSITY', 'MISSION COLLEGE', 'MISSISSIPPI COLLEGE', 'MISSISSIPPI DELTA COMMUNITY COLLEGE (PRIMARY)', 'MISSISSIPPI GULF COAST COMMUNITY COLLEGE (PRIMARY)', 'MISSISSIPPI STATE UNIVERSITY', 'MISSISSIPPI UNIVERSITY FOR WOMEN', 'MISSISSIPPI VALLEY STATE UNIVERSITY', 'MISSOURI BAPTIST UNIVERSITY', 'MISSOURI COLLEGE', 'MISSOURI SOUTHERN STATE UNIVERSITY', 'MISSOURI STATE UNIVERSITY', 'MISSOURI STATE UNIVERSITY - WEST PLAINS', 'MISSOURI UNIVERSITY SCIENCE TECHNOLOGY', 'MISSOURI VALLEY COLLEGE', 'MISSOURI WESTERN STATE UNIVERSITY', 'MITCHELL COLLEGE', 'MITCHELL COMMUNITY COLLEGE', 'MITCHELL TECHNICAL COLLEGE', 'MOBERLY AREA COMMUNITY COLLEGE', 'MODESTO JUNIOR COLLEGE', 'MOHAVE COMMUNITY COLLEGE', 'MOHAWK VALLEY COMMUNITY COLLEGE', 'MOLLOY COLL', 'MONMOUTH COLLEGE', 'MONMOUTH UNIVERSITY', 'MONROE COLLEGE', 'MONROE COMMUNITY COLLEGE', 'MONROE COMMUNITY COLLEGE (NY)', 'MONROE COUNTY COMMUNITY COLLEGE (MI)', 'MONTANA ST UNIV C TECH-GREAT FALLS', 'MONTANA STATE UNIVERSITY', 'MONTANA STATE UNIVERSITY BILLINGS', 'MONTANA STATE UNIVERSITY-NORTHERN', 'MONTANA TECHNOLOGICAL UNIVERSITY', 'MONTCALM COMMUNITY COLLEGE', 'MONTCLAIR STATE UNIVERSITY', 'MONTEREY PENINSULA COLLEGE', 'MONTGOMERY COLLEGE', 'MONTGOMERY COMMUNITY COLLEGE', 'MONTGOMERY COUNTY COMMUNITY COLLEGE', 'MONTREAT COLLEGE', 'MONTSERRAT COLLEGE ART', 'MOODY BIBLE INST', 'MOORE COLLEGE ART DESIGN', 'MOORPARK COLLEGE', 'MORAINE PARK TECHNICAL COLLEGE', 'MORAINE VALLEY COMMUNITY COLLEGE', 'MORAVIAN COLL', 'MOREHEAD STATE UNIVERSITY (PRIMARY)', 'MOREHOUSE COLLEGE', 'MORENO VALLEY COLLEGE', 'MORGAN COMMUNITY COLLEGE', 'MORGAN STATE UNIVERSITY', 'MORNINGSIDE COLLEGE', 'MORRISVILLE STATE COLLEGE', 'MORTON COLL', 'MOTLOW STATE COMMUNITY COLLEGE', 'MOTT COMMUNITY COLLEGE', 'MOUNT ALOYSIUS COLLEGE', 'MOUNT CARMEL COLLEGE NURSING', 'MOUNT HOLYOKE COLLEGE', 'MOUNT HOOD COMMUNITY COLLEGE', 'MOUNT IDA COLLEGE', 'MOUNT MARTY COLLEGE (PRIMARY)', 'MOUNT MARY UNIVERSITY', 'MOUNT MERCY UNIVERSITY', 'MOUNT ROYAL UNIVERSITY', 'MOUNT SAINT MARY COL', "MOUNT SAINT MARY'S COLLEGE", 'MOUNT SAN ANTONIO COLLEGE', 'MOUNT SAN JACINTO COLLEGE', 'MOUNT ST MARYS C MD', 'MOUNT ST. JOSEPH UNIVERSITY', "MOUNT ST. MARY'S COLLEGE", 'MOUNT VERNON NAZARENE UNIVERSITY', 'MOUNT WACHUSETT COMMUNITY COLLEGE', 'MOUNTAIN EMPIRE COMMUNITY COLLEGE', 'MOUNTAIN GATEWAY COMMUNITY COLLEGE', 'MOUNTAIN STATE UNIVERSITY', 'MOUNTWEST COMM TECH COLL', 'MTI COLLEGE', 'MUAP-111', 'MUHLENBERG COLL', 'MURRAY STATE COLLEGE (OK) - PRIMARY', 'MURRAY STATE UNIVERSITY', 'MUS-344', 'MUSKEGON COMMUNITY COLLEGE', 'MUSKINGUM UNIVERSITY', 'NAMIBIA UNIVERSITY SCIENCE TECHNOLOGY', 'NAPA VALLEY COLLEGE', 'NASH COMMUNITY COLLEGE', 'NASHUA COMMUNITY COLLEGE', 'NASHVILLE STATE COMMUNITY COLLEGE', 'NASSAU COMMUNITY COLLEGE', 'NATIONAL AMERICAN UNIVERSITY', 'NATIONAL PARK COLLEGE', 'NATIONAL UNIV HEALTH SCIENCES', 'NATIONAL UNIVERSITY', 'NATIONAL-LOUIS UNIVERSITY', 'NAUGATUCK VALLEY COMMUNITY COLLEGE', 'NAVARRO COLLEGE', 'NAZARETH COLLEGE ROCHESTER', 'NAZARETH UNIVERSITY', 'NEBRASKA COLLEGE TECHNICAL AGRICULTURE', 'NEBRASKA METHODIST COLLEGE', 'NEBRASKA WESLEYAN UNIVERSITY', 'NEOSHO COUNTY COMMUNITY COLLEGE', 'NEUMANN UNIVERSITY', 'NEVADA STATE COLLEGE', 'NEW COLLEGE FLORIDA', 'NEW ENGLAND COLLEGE', 'NEW ENGLAND INSTITUTE ART', 'NEW ENGLAND INSTITUTE TECHNOLOGY', 'NEW JERSEY CITY UNIVERSITY', 'NEW JERSEY INSTITUTE TECHNOLOGY', 'NEW MEXICO HIGHLANDS UNIV', 'NEW MEXICO JUNIOR COLLEGE', 'NEW MEXICO MILITARY INSTITUTE', 'NEW MEXICO ST UNIV ALAMOGORDO', 'NEW MEXICO TECH', 'NEW ORLEANS BAPT THEO SEM', 'NEW RIVER COMMUNITY COLLEGE', 'NEW RIVER COMMUNITY TECHNICAL COLLEGE (PRIMARY)', 'NEW SCHOOL', 'NEW YORK CITY COLLEGE TECHNOLOGY', 'NEW YORK FILM ACADEMY', 'NEW YORK INSTITUTE TECHNOLOGY', 'NEW YORK UNIVERSITY', 'NEWBERRY COLLEGE', 'NEWBURY COLLEGE', 'NEWMAN UNIVERSITY', "NHTI - CONCORD'S COMMUNITY COLLEGE", 'NIAGARA COUNTY COMMUNITY COLLEGE', 'NIAGARA UNIV', 'NICHOLLS ST UNIV', 'NICOLET AREA TECHNICAL COLLEGE', 'NIGHTINGALE COLLEGE', 'NORCO COLLEGE', 'NORFOLK STATE UNIVERSITY', 'NORMANDALE COMMUNITY COLLEGE', 'NORTH ARKANSAS COLLEGE', 'NORTH CAROLINA A&T STATE UNIVERSITY', 'NORTH CAROLINA CENTRAL UNIVERSITY', 'NORTH CAROLINA STATE UNIVERSITY AT RALEIGH', 'NORTH CAROLINA WESLEYAN COLLEGE', 'NORTH CENTRAL COLLEGE', 'NORTH CENTRAL KANSAS TECHNICAL COLLEGE', 'NORTH CENTRAL MICHIGAN COLLEGE', 'NORTH CENTRAL MISSOURI COLLEGE', 'NORTH CENTRAL STATE COLLEGE', 'NORTH CENTRAL TEXAS COLLEGE', 'NORTH CENTRAL UNIVERSITY', 'NORTH COUNTRY C C', 'NORTH DAKOTA STATE COLLEGE SCIENCE', 'NORTH DAKOTA STATE UNIVERSITY', 'NORTH FLORIDA JR COL', 'NORTH GEORGIA TECHNICAL COLLEGE', 'NORTH GREENVILLE UNIVERSITY', 'NORTH HENNEPIN COMMUNITY COLLEGE', 'NORTH IDAHO COLLEGE', 'NORTH IOWA AREA COMMUNITY COLLEGE', 'NORTH PARK UNIVERSITY', 'NORTH SEATTLE C C', 'NORTH SHORE COMMUNITY COLLEGE', 'NORTHAMPTON COUNTY AREA COMMUNITY COLLEGE', 'NORTHCENTRAL TECHNICAL COLLEGE', 'NORTHEAST ALABAMA COMMUNITY COLLEGE', 'NORTHEAST COMMUNITY COLLEGE', 'NORTHEAST IOWA COMMUNITY COLLEGE (PRIMARY)', 'NORTHEAST LAKEVIEW COLLEGE', 'NORTHEAST MISS JR C', 'NORTHEAST STATE COMMUNITY COLLEGE', 'NORTHEAST TEXAS COMM COLLEGE', 'NORTHEAST WISCONSIN TECHNICAL COLLEGE', 'NORTHEASTERN ILLINOIS UNIVERSITY', 'NORTHEASTERN JUNIOR COLLEGE', 'NORTHEASTERN OKLAHOMA A&M COLLEGE', 'NORTHEASTERN STATE UNIVERSITY (OK)', 'NORTHEASTERN TECHNICAL COLLEGE', 'NORTHEASTERN UNIVERSITY', 'NORTHERN ARIZONA UNIVERSITY', 'NORTHERN ESSEX COMMUNITY COLLEGE', 'NORTHERN ILLINOIS UNIVERSITY', 'NORTHERN KENTUCKY UNIVERSITY', 'NORTHERN MAINE COMMUNITY COLLEGE', 'NORTHERN MARIANAS COLLEGE', 'NORTHERN MICHIGAN UNIVERSITY', 'NORTHERN NEW MEXICO COLLEGE', 'NORTHERN OKLAHOMA COLLEGE (PRIMARY)', 'NORTHERN STATE UNIVERSITY', 'NORTHERN VIRGINIA COMMUNITY COLLEGE', 'NORTHERN WYOMING COMMUNITY COLLEGE DISTRICT (PRIMARY)', 'NORTHLAND COLLEGE', 'NORTHLAND COMMUNITY TECHNICAL COLLEGE (PRIMARY)', 'NORTHLAND PIONEER COLLEGE', 'NORTHSHORE TECHNICAL COMMUNITY COLLEGE (PRIMARY)', 'NORTHSIDE HIGH SCHOOL', 'NORTHWEST ARKANSAS COMMUNITY COLLEGE', 'NORTHWEST COLLEGE', 'NORTHWEST FLORIDA STATE COLLEGE', 'NORTHWEST INDIAN COLLEGE', 'NORTHWEST IOWA COMMUNITY COLLEGE', 'NORTHWEST MISSISSIPPI COMMUNITY COLLEGE (PRIMARY)', 'NORTHWEST MISSOURI STATE UNIVERSITY', 'NORTHWEST NAZARENE UNIVERSITY', 'NORTHWEST STATE COMMUNITY COLLEGE', 'NORTHWEST TECHNICAL COLLEGE', 'NORTHWEST VISTA COLLEGE', 'NORTHWEST-SHOALS COMMUNITY COLLEGE (PRIMARY)', 'NORTHWESTERN COLL-BRIDGEVIEW', 'NORTHWESTERN COLLEGE', 'NORTHWESTERN CONNECTICUT CC', 'NORTHWESTERN HEALTH SCI UNIV', 'NORTHWESTERN MICHIGAN COLLEGE', 'NORTHWESTERN OKLAHOMA STATE UNIVERSITY (PRIMARY)', 'NORTHWESTERN STATE UNIVERSITY LOUISIANA (PRIMARY)', 'NORTHWESTERN UNIVERSITY', 'NORTHWOOD TECHNICAL COLLEGE (PRIMARY)', 'NORTHWOOD UNIVERSITY (PRIMARY)', 'NORWALK COMMUNITY COLLEGE', 'NORWICH UNIVERSITY', 'NOTRE DAME COLLEGE (MO)', 'NOTRE DAME COLLEGE OHIO', 'NOTRE DAME DE NAMUR UNIVERSITY', 'NOTRE DAME MARYLAND UNIVERSITY', 'NOVA SOUTHEASTERN UNIVERSITY', 'NUC UNIVERSITY (PRIMARY)', 'NUNEZ COMMUNITY COLLEGE', 'OAK POINT UNIVERSITY', 'OAKLAND CITY COLL', 'OAKLAND COMMUNITY COLLEGE', 'OAKLAND UNIVERSITY', 'OAKTON COMMUNITY COLLEGE', 'OAKWOOD UNIVERSITY', 'OBERLIN COLLEGE', 'OCCIDENTAL COLL', 'OCEAN COUNTY COLLEGE', 'OCONEE FALL LINE TECHNICAL COLL', 'ODESSA COLLEGE', 'OGDEN-WEBER TECHNICAL COLLEGE', 'OGLALA LAKOTA COLLEGE', 'OGLETHORPE UNIV', 'OHIO CHRISTIAN UNIVERSITY', 'OHIO DOMINICAN UNIVERSITY', 'OHIO NORTHERN UNIV', 'OHIO STATE UNIVERSITY (PRIMARY)', 'OHIO UNIVERSITY (PRIMARY)', 'OHIO VALLEY UNIVERSITY', 'OHIO WESLEYAN UNIVERSITY', 'OHLONE COLLEGE', 'OKANAGAN COLLEGE', 'OKLAHOMA BAPTIST UNIVERSITY', 'OKLAHOMA CHRISTIAN UNIVERSITY', 'OKLAHOMA CITY COMMUNITY COLLEGE', 'OKLAHOMA CITY UNIVERSITY', 'OKLAHOMA JUNIOR COLLEGE BUSINESS TECHNOLOGY', 'OKLAHOMA PANHANDLE STATE UNIVERSITY', 'OKLAHOMA ST UNIV-OKMULGEE', 'OKLAHOMA STATE UNIVERSITY', 'OKLAHOMA STATE UNIVERSITY - OKLAHOMA CITY', 'OKLAHOMA WESLEYAN UNIVERSITY', 'OLD DOMINION UNIVERSITY', 'OLIVET COLLEGE', 'OLIVET NAZARENE UNIVERSITY', 'OLYMPIC COLLEGE', 'ONONDAGA COMMUNITY COLLEGE', 'ORAL ROBERTS UNIVERSITY', 'ORANGE COAST COLL', 'ORANGE COUNTY COMMUNITY COLLEGE', 'ORANGEBURG-CALHOUN TECH COLLEGE', 'OREGON HEALTH SCIENCE UNIVERSITY', 'OREGON INSTITUTE TECHNOLOGY', 'OREGON STATE UNIVERSITY', 'OTERO JR COLL', 'OTTAWA UNIVERSITY', 'OTTERBEIN UNIVERSITY', 'OUACHITA BAPTIST UNIVERSITY', 'OUR LADY HOLY CROSS', 'OUR LADY LAKE UNIV', 'OWENS COMMUNITY COLLEGE (PRIMARY)', 'OXFORD BROOKES UNIV', 'OXFORD COLL EMORY UNIVERSITY', 'OXNARD COLLEGE', 'OXON HILL SENIOR HIGH SCHOOL', 'OZARK CHRISTIAN COLLEGE', 'OZARKA COLLEGE', 'OZARKS TECHNICAL COMMUNITY COLLEGE', 'PACE UNIVERSITY (PRIMARY)', 'PACIFIC LUTHERAN UNIVERSITY', 'PACIFIC OAKS COLLEGE', 'PACIFIC UNIVERSITY', 'PAINE COLLEGE', 'PALM BEACH ATLANTIC UNIVERSITY', 'PALM BEACH STATE COLLEGE', 'PALO ALTO COLLEGE', 'PALO VERDE COLL', 'PALOMAR COLLEGE', 'PANAMA CANAL COLLEGE', 'PANOLA COLLEGE', 'PAPD-102', 'PARIS JR COLL', 'PARK UNIVERSITY', 'PARKER UNIVERSITY', 'PARKLAND COLLEGE', 'PASADENA CITY COLLEGE', 'PASCO-HERNANDO STATE COLLEGE', 'PASM-121', 'PASSAIC COUNTY COMMUNITY COLLEGE', 'PATRICK HENRY COMMUNITY COLLEGE', 'PAUL D CAMP C C', 'PAUL QUINN COLLEGE', "PAUL SMITH'S COLLEGE", 'PE-120', 'PEABODY INSTITUTE', 'PEARL RIVER COMMUNITY COLLEGE (PRIMARY)', 'PEGN-210', 'PELLISSIPPI STATE COMMUNITY COLLEGE (PRIMARY)', 'PENINSULA COLL', 'PENN FOSTER CAREER SCHOOL', 'PENNSYLVANIA COLLEGE HEALTH SCIENCES', 'PENNSYLVANIA COLLEGE TECHNOLOGY', 'PENNSYLVANIA HIGHLANDS COMMUNITY COLLEGE', 'PENNSYLVANIA STATE UNIVERSITY', 'PENNSYLVANIA WESTERN UNIVERSITY (PRIMARY)', 'PENSACOLA CHRISTIAN COLLEGE', 'PENSACOLA STATE COLLEGE', 'PERU STATE COLLEGE', 'PFEIFFER UNIVERSITY', 'PHILADELPHIA UNIVERSITY', 'PHILANDER SMITH COLLEGE', 'PHILLIPS 66', 'PHILLIPS COMMUNITY COLLEGE', 'PIEDMONT COLLEGE', 'PIEDMONT COMMUNITY COLLEGE', 'PIEDMONT TECHNICAL COLLEGE', 'PIEDMONT VIRGINIA COMMUNITY COLLEGE', 'PIERCE COLLEGE', 'PIERPONT COMM TECH COLLEGE', 'PIKES PEAK COMMUNITY COLLEGE', 'PIMA COMMUNITY COLLEGE', 'PINE TECHNICAL COLLEGE', 'PITT COMMUNITY COLLEGE', 'PITTSBURG STATE UNIVERSITY', 'PITTSBURGH TECHNICAL INSTITUTE', 'PITZER COLLEGE', 'PLATT COLLEGE', 'PLATT COLLEGE (OKLAHOMA)', 'PLAZA COLLEGE', 'PLYMOUTH STATE UNIVERSITY', 'POINT LOMA NAZARENE UNIVERSITY', 'POINT PARK UNIVERSITY', 'POINT UNIVERSITY', 'POLK STATE COLLEGE', 'PONTIFICAL CATHOLIC UNIVERSITY PUERTO RICO', 'PONTIFICIA UNIVERSIDAD CATOLICA MADRE Y MAESTRA', 'PORTERVILLE COLLEGE', 'PORTLAND COMMUNITY COLLEGE', 'PORTLAND STATE UNIVERSITY', 'POST UNIVERSITY', 'POTOMAC STATE COLLEGE WEST VIRGINIA UNIVERSITY', 'PRAIRIE STATE COLLEGE', 'PRAIRIE VIEW A&M UNIVERSITY', 'PRATT COMMUNITY COLLEGE', 'PRATT INSTITUTE', 'PRESBYTERIAN COLLEGE', 'PRESCOTT COLLEGE', 'PRESENTATION COLLEGE (PRIMARY)', "PRINCE GEORGE'S COMMUNITY COLLEGE", 'PRINCETON UNIVERSITY', 'PRINCIPIA COLLEGE', 'PROVIDENCE COLLEGE', 'PUEBLO COMMUNITY COLLEGE (PRIMARY)', 'PUERTO RICO JUNIOR COLLEGE', 'PULASKI TECHNICAL COLLEGE', 'PURDUE UNIV CALUMET', 'PURDUE UNIVERSITY FORT WAYNE', 'PURDUE UNIVERSITY GLOBAL (FORMERLY KAPLAN UNIVERSITY)', 'PURDUE UNIVERSITY NORTHWEST - WESTVILLE', 'PURDUE UNIVERSITY WEST LAFAYETTE', 'QUEENS COLLEGE NY', 'QUEENS UNIVERSITY CHARLOTTE', 'QUEENSBOROUGH COMMUNITY COLLEGE', 'QUINCY COLLEGE', 'QUINCY UNIVERSITY', 'QUINEBAUG VALLEY COMMUNITY COLLEGE', 'QUINNIPIAC UNIVERSITY', 'QUINSIGAMOND COMMUNITY COLLEGE', 'RADFORD UNIVERSITY', 'RAINY RIVER COMMUNITY COLLEGE', 'RAMAPO COLLEGE NEW JERSEY', 'RANDOLPH COLLEGE', 'RANDOLPH COMMUNITY COLLEGE', 'RANDOLPH MACON COLL', 'RANGER COLLEGE', 'RANKEN TECHNICAL COLLEGE', 'RAPPAHANNOCK COMMUNITY COLLEGE (PRIMARY)', 'RARITAN VALLEY COMMUNITY COLLEGE', 'RASMUSSEN COLLEGE (PRIMARY)', 'READING AREA C C', 'RED ROCKS COMMUNITY COLLEGE', 'REDLANDS COMMUNITY COLLEGE', 'REEDLEY COLLEGE', 'REGENT UNIVERSITY', 'REGIS COLLEGE', 'REGIS UNIVERSITY', 'REINHARDT UNIVERSITY', 'REND LAKE COLLEGE', 'RENSSELAER POLY INST', 'RENTON TECHNICAL COLLEGE', 'RESEARCH COLLEGE NURSING', 'RHODE ISLAND COLL', 'RHODES COLLEGE', 'RHODES STATE COLLEGE', 'RICE UNIVERSITY', 'RICH MOUNTAIN COMMUNITY COLLEGE', 'RICHARD BLAND COLLEGE WILLIAM MARY', 'RICHLAND COMMUNITY COLLEGE', 'RICHMOND AMERICAN INTL U IN LONDON', 'RICHMOND COMMUNITY COLLEGE', 'RIDER UNIVERSITY', 'RIDGEWATER COLLEGE', 'RIO HONDO COLLEGE', 'RIPON COLLEGE', 'RIVER PARISHES COMMUNITY COLLEGE', 'RIVER VALLEY COMMUNITY COLLEGE', 'RIVERLAND COMMUNITY COLLEGE', 'RIVERSIDE CITY COLLEGE', 'RIVIER UNIVERSITY', 'RN LICENSE', 'ROANE STATE COMMUNITY COLLEGE (PRIMARY)', 'ROANOKE COLLEGE', 'ROANOKE-CHOWAN COMMUNITY COLLEGE', 'ROBERT B. MILLER COLLEGE', 'ROBERT MORRIS UNIVERSITY', 'ROBERTS WESLEYAN COL', 'ROBESON COMMUNITY COLLEGE', 'ROCHESTER COMMUNITY TECHNICAL COLLEGE', 'ROCHESTER INSTITUTE TECHNOLOGY', 'ROCHESTER UNIVERSITY', 'ROCK VALLEY COLLEGE', 'ROCKFORD UNIVERSITY', 'ROCKHURST UNIVERSITY', 'ROCKINGHAM COMMUNITY COLLEGE', 'ROCKLAND COMMUNITY COLLEGE', 'ROCKY MOUNTAIN COLL', 'ROCKY MOUNTAIN COLLEGE ART DESIGN', 'ROCKY MOUNTAIN UNIVERSITY HEALTH PROFESSIONS', 'ROGER WILLIAMS UNIVERSITY', 'ROGERS STATE UNIVERSITY', 'ROGUE COMMUNITY COLLEGE', 'ROOSEVELT UNIVERSITY', 'ROSE STATE COLLEGE', 'ROSE-HULMAN INSTITUTE TECHNOLOGY', 'ROWAN COLLEGE AT BURLINGTON COUNTY', 'ROWAN COLLEGE SOUTH JERSEY', 'ROWAN UNIVERSITY', 'ROWAN-CABARRUS COMMUNITY COLLEGE', 'ROXBURY COMMUNITY COLLEGE', 'RUSH UNIVERSITY', 'RUSSELL SAGE COLLEGE', 'RUTGERS UNIVERSITY', 'SACRAMENTO CITY COLLEGE', 'SACRED HEART UNIVERSITY', 'SADDLEBACK COLLEGE', 'SAGE COLLEGE ALBANY', 'SAGINAW VALLEY STATE UNIVERSITY', 'SAINT AUGUSTINE COLLEGE', 'SAINT CLOUD STATE UNIVERSITY', 'SAINT FRANCIS MEDICAL CENTER COLLEGE NURSING', "SAINT JOHN'S UNIVERSITY", "SAINT JOSEPH'S COLLEGE (NY) - PRIMARY", "SAINT JOSEPH'S COLLEGE MAINE", "SAINT JOSEPH'S UNIVERSITY", 'SAINT LEO UNIVERSITY', 'SAINT LOUIS UNIVERSITY', "SAINT MARTIN'S UNIVERSITY", "SAINT MARY'S COLLEGE O'FALLON", "SAINT MARY'S UNIVERSITY", "SAINT MARY'S UNIVERSITY MINNESOTA", 'SAINT MARY-OF-THE-WOODS COLLEGE', 'SAINT MEINRAD SCHOOL THEOLOGY', 'SAINT MICHAELS COLLEGE', 'SAINT OLAF COLLEGE', 'SAINT PAUL COLLEGE', "SAINT PETER'S UNIVERSITY", 'SAINT XAVIER UNIVERSITY', 'SALEM COLLEGE', 'SALEM COMMUNITY COLLEGE', 'SALEM STATE UNIVERSITY', 'SALEM UNIVERSITY', 'SALINA AREA TECHNICAL COLLEGE', 'SALISBURY UNIVERSITY', 'SALT LAKE COMMUNITY COLLEGE', 'SALVE REGINA COLLEGE', 'SAM HOUSTON STATE UNIVERSITY', 'SAMFORD UNIVERSITY', 'SAMPSON COMMUNITY COLLEGE', 'SAMUEL MERRITT UNIVERSITY', 'SAN ANTONIO COLLEGE', 'SAN BERNARDINO VALLEY COLLEGE', 'SAN DIEGO CHRISTIAN COLLEGE', 'SAN DIEGO COMMUNITY COLLEGE DISTRICT (PRIMARY)', 'SAN DIEGO STATE UNIVERSITY', 'SAN FRANCISCO STATE UNIVERSITY', 'SAN JACINTO COMMUNITY COLLEGE DISTRICT (PRIMARY)', 'SAN JOAQUIN DELTA COLLEGE', 'SAN JOAQUIN VALLEY COLLEGE', 'SAN JOSE CITY COLLEGE', 'SAN JOSE STATE UNIVERSITY', 'SAN JUAN COLLEGE', 'SANDHILLS COMMUNITY COLLEGE', 'SANFORD BROWN COLLEGE', 'SANFORD-BROWN COLLEGE-BOSTON', 'SANTA ANA COLLEGE', 'SANTA BARBARA CITY COLLEGE', 'SANTA CLARA UNIVERSITY', 'SANTA FE COLLEGE', 'SANTA FE COMMUNITY COLLEGE', 'SANTA FE UNIV ART DESIGN', 'SANTA MONICA COLLEGE', 'SANTA ROSA JUNIOR COLLEGE', 'SANTIAGO COLLEGE', 'SAUK VALLEY COMMUNITY COLLEGE', 'SAVANNAH COLLEGE ART DESIGN', 'SAVANNAH STATE UNIVERSITY', 'SAVANNAH TECHNICAL COLLEGE', 'SCHALMONT SR HS', 'SCHENECTADY COUNTY COMMUNITY COLLEGE', 'SCHOOL ART INSTITUTE CHICAGO', 'SCHOOLCRAFT COLLEGE', 'SEATTLE CENTRAL COLLEGE', 'SEATTLE PACIFIC UNIVERSITY', 'SEATTLE UNIVERSITY', 'SEMINOLE STATE COLLEGE (OK)', 'SEMINOLE STATE COLLEGE FLORIDA', 'SETON HALL UNIVERSITY', 'SETON HILL UNIVERSITY', 'SEWARD COUNTY COMMUNITY COLLEGE', 'SHASTA COLLEGE', 'SHAW UNIVERSITY', 'SHAWNEE COMMUNITY COLLEGE', 'SHAWNEE STATE UNIVERSITY', 'SHELDON JACKSON COLLEGE', 'SHELTON STATE COMMUNITY COLLEGE', 'SHENANDOAH UNIVERSITY', 'SHEPHERD UNIVERSITY', 'SHIPPENSBURG UNIVERSITY PENNSYLVANIA', 'SHORELINE COMMUNITY COLLEGE', 'SHORTER UNIVERSITY', 'SIENA COLLEGE', 'SIENA HEIGHTS UNIVERSITY', 'SIERRA COLLEGE', 'SIERRA NEVADA COLLEGE', 'SIMMONS UNIVERSITY', 'SIMON FRASER UNIVERSITY', 'SIMPSON COLLEGE', 'SIMPSON UNIVERSITY', 'SINCLAIR COMMUNITY COLLEGE', 'SINTE GLESKA UNIVERSITY', 'SKAGIT VALLEY COLLEGE', 'SKIDMORE COLLEGE', 'SKYLINE COLLEGE', 'SLIPPERY ROCK UNIVERSITY PENNSYLVANIA', 'SNEAD STATE COMMUNITY COLLEGE', 'SNOW COLLEGE', 'SOC-101', 'SOLANO COMMUNITY COLLEGE', 'SONOMA STATE UNIVERSITY', 'SOPHIA LEARNING', 'SOUTH ARKANSAS COMMUNITY COLLEGE', 'SOUTH CENTRAL COLLEGE (PRIMARY)', 'SOUTH COLLEGE-KNOXVILLE', 'SOUTH DAKOTA SCHOOL MINES TECHNOLOGY', 'SOUTH DAKOTA STATE UNIVERSITY', 'SOUTH FLORIDA STATE COLLEGE', 'SOUTH GEORGIA STATE COLLEGE', 'SOUTH GEORGIA TECHNICAL COLLEGE', 'SOUTH LOUISIANA COMMUNITY COLLEGE (PRIMARY)', 'SOUTH PIEDMONT COMMUNITY COLLEGE', 'SOUTH PLAINS COLLEGE', 'SOUTH PUGET SOUND COMMUNITY COLLEGE', 'SOUTH SEATTLE COMMUNITY COLLEGE', 'SOUTH SUBURBAN COLLEGE', 'SOUTH TEXAS COLLEGE', 'SOUTH UNIVERSITY (PRIMARY)', 'SOUTHEAST ARKANSAS COLLEGE', 'SOUTHEAST COMMUNITY COLLEGE (NE) - PRIMARY', 'SOUTHEAST MISSOURI HOSPITAL COLLEGE NURSING HEALTH SCI', 'SOUTHEAST MISSOURI STATE UNIVERSITY', 'SOUTHEAST TECHNICAL INSTITUTE', 'SOUTHEASTERN BAPTIST THEOL. SEM', 'SOUTHEASTERN COMMUNITY COLLEGE', 'SOUTHEASTERN COMMUNITY COLLEGE (IA) - PRIMARY', 'SOUTHEASTERN ILLINOIS COLLEGE', 'SOUTHEASTERN LOUISIANA UNIV.', 'SOUTHEASTERN OKLAHOMA STATE UNIVERSITY (PRIMARY)', 'SOUTHEASTERN TECHNICAL COLLEGE', 'SOUTHEASTERN UNIVERSITY (FL)', 'SOUTHERN ADVENTIST UNIVERSITY', 'SOUTHERN ALBERTA INSTITUTE TECHNOLOGY', 'SOUTHERN ARKANSAS UNIV TECH', 'SOUTHERN ARKANSAS UNIVERSITY', 'SOUTHERN CALIFORNIA UNIVERSITY HEALTH SCIENCES', 'SOUTHERN CONNECTICUT STATE UNIVERSITY', 'SOUTHERN CRESCENT TECHNICAL COLLEGE (PRIMARY)', 'SOUTHERN ILLINOIS UNIVERSITY - CARBONDALE', 'SOUTHERN ILLINOIS UNIVERSITY - EDWARDSVILLE', 'SOUTHERN MAINE COMMUNITY COLLEGE', 'SOUTHERN METHODIST COLLEGE', 'SOUTHERN METHODIST UNIVERSITY', 'SOUTHERN NAZARENE UNIVERSITY', 'SOUTHERN NEW HAMPSHIRE UNIVERSITY', 'SOUTHERN OREGON UNIVERSITY', 'SOUTHERN REGIONAL TECHNICAL COLLEGE', 'SOUTHERN STATE COMMUNITY COLLEGE', 'SOUTHERN UNION STATE COMMUNITY COLLEGE', 'SOUTHERN UNIV AT SHREVEPORT', 'SOUTHERN UNIVERSITY A&M COLLEGE', 'SOUTHERN UNIVERSITY AT NEW ORLEANS', 'SOUTHERN UTAH UNIVERSITY', 'SOUTHERN VERMONT COLLEGE', 'SOUTHERN VIRGINIA UNIVERSITY', 'SOUTHERN WESLEYAN UNIV', 'SOUTHERN WEST VIRGINIA COMMUNITY TECHNICAL COLLEGE (P)', 'SOUTHSIDE VIRGINIA COMMUNITY COLLEGE', 'SOUTHWEST BAPTIST UNIVERSITY (PRIMARY)', 'SOUTHWEST MINNESOTA STATE UNIVERSITY', 'SOUTHWEST MISSISSIPPI COMMUNITY COLLEGE', 'SOUTHWEST TENNESSEE COMMUNITY COLLEGE (PRIMARY)', 'SOUTHWEST TEXAS JR COLLEGE', 'SOUTHWEST VIRGINIA COMMUNITY COLLEGE', 'SOUTHWEST WISCONSIN TECHNICAL COLLEGE', 'SOUTHWESTERN ADVENTIST UNIVERSITY', 'SOUTHWESTERN ASSEMBLIES GOD UNIV', 'SOUTHWESTERN COLLEGE', 'SOUTHWESTERN COLLEGE (CA)', 'SOUTHWESTERN COMMUNITY COLLEGE', 'SOUTHWESTERN COMMUNITY COLLEGE (IA)', 'SOUTHWESTERN ILLINOIS COLLEGE', 'SOUTHWESTERN MICHIGAN COLLEGE', 'SOUTHWESTERN OKLAHOMA STATE UNIVERSITY (PRIMARY)', 'SOUTHWESTERN OREGON COMMUNITY COLLEGE', 'SOUTHWESTERN UNIVERSITY', 'SOWELA TECHNICAL COMMUNITY COLLEGE', 'SP X-210', 'SPALDING UNIVERSITY', 'SPARTANBURG COMMUNITY COLLEGE', 'SPARTANBURG METHODIST COLL', 'SPELMAN COLLEGE', 'SPH-110', 'SPOKANE COMMUNITY COLLEGE', 'SPOKANE FALLS COMMUNITY COLLEGE', 'SPOON RIVER COLLEGE', 'SPRING ARBOR UNIVERSITY', 'SPRING HILL COLLEGE', 'SPRINGFIELD COLL', 'SPRINGFIELD COLL ILL', 'SPRINGFIELD TECHNICAL COMMUNITY COLLEGE', 'SPRINT PCS', 'ST ANDREWS UNIVERSITY', 'ST ANTHONY SCH NURS', 'ST CATHARINE COLL', 'ST EDWARDS UNIV', 'ST JOHN FISHER COLL', "ST JOHN'S COLLEGE JUNIOR COLL DIV", 'ST JOSEPH HOSPITAL SCHOOL NURSING', 'ST LAWRENCE UNIV', "ST MARY'S DOM HS", "ST MARYS COLLEGE-O'FALLON", 'ST NORBERT COLL', 'ST PETERSBURG COLLEGE (PRIMARY)', "ST PHILIP'S COLLEGE", 'ST THOMAS AQUINAS C', 'ST THOMAS UNIVERSITY', 'ST VINCENT COLL', 'ST. AMBROSE UNIVERSITY', 'ST. ANDREWS UNIVERSITY', "ST. AUGUSTINE'S UNIVERSITY", 'ST. BONAVENTURE UNIVERSITY', 'ST. CATHERINE UNIVERSITY', 'ST. CHARLES COMMUNITY COLLEGE', 'ST. CLAIR COUNTY COMMUNITY COLLEGE', 'ST. CLOUD TECHNICAL COMMUNITY COLLEGE', 'ST. FRANCIS COLLEGE (NY)', "ST. JOHN'S RIVER STATE COLLEGE", "ST. JOHN'S UNIVERSITY (NY)", "ST. JOSEPH'S COLLEGE (IN)", 'ST. JOSEPHS UNIVERSITY (PA)', 'ST. LOUIS COMMUNITY COLLEGE (PRIMARY)', "ST. MARY'S COLLEGE", "ST. MARY'S COLLEGE CALIFORNIA", "ST. MARY'S UNIVERSITY (TX)", 'STANFORD UNIVERSITY', 'STANLY COMMUNITY COLLEGE', 'STARK STATE COLLEGE', 'STATE COLLEGE FLORIDA MANATEE-SARASOTA', 'STATE COMMUNITY COLLEGE EAST SAINT LOUIS', 'STATE FAIR COMMUNITY COLLEGE', 'STATE TECHNICAL COLLEGE MISSOURI', 'STATE UNIV NEW YORK AT NEW PALTZ', 'STATE UNIV NY COLL ENVIRON/FORESTRY', 'STATE UNIV NY UPSTATE MEDICAL UNIV', 'STATE UNIVERSITY NEW YORK - ADIRONDACK', 'STATE UNIVERSITY NEW YORK AT GENESEO', 'STATE UNIVERSITY NEW YORK COLLEGE AT PLATTSBURGH', 'STATE UNIVERSITY NY COLLEGE TECHNOLOGY AT DELHI', 'STATE UNIVOF NEW YORK OSWEGO', 'STEPHEN F AUSTIN STATE UNIVERSITY', 'STEPHENS COLLEGE', 'STERLING COLLEGE (KS)', 'STETSON UNIVERSITY', 'STEVENS INST BUSINESS ARTS', 'STEVENSON UNIVERSITY', 'STILLMAN COLLEGE', 'STOCKTON UNIVERSITY', 'STONE CHILD COLLEGE', 'STONEHILL COLL', 'STONY BROOK UNIVERSITY', 'STRAIGHTERLINE UNIVERSITY', 'STRAYER UNIVERSITY', 'STUDY. COM', 'SUE BENNETT COLLEGE', 'SUFFOLK COUNTY COMMUNITY COLLEGE', 'SUFFOLK UNIVERSITY', 'SUL ROSS ST UNIV', 'SULLIVAN COUNTY COMMUNITY COLLEGE', 'SULLIVAN UNIVERSITY', 'SUNY AGR TEC C FARMG', 'SUNY AT ALBANY', 'SUNY AT FREDONIA', 'SUNY CANTON COLLEGE TECHNOLOGY', 'SUNY COLL BROCKPORT', 'SUNY COLL POTSDAM', 'SUNY COLL PURCHASE', 'SUNY COLLEGE AGRICULTURE TECHNOLOGY AT COBLESKILL', 'SUNY COLLEGE AT CORTLAND', 'SUNY COLLEGE AT OLD WESTBURY', 'SUNY COLLEGE AT ONEONTA', 'SUNY DOWNSTATE HEALTH SCIENCES UNIVERSITY', 'SUNY EMPIRE STATE COLLEGE', 'SUNY POLYTECHNIC INSTITUTE', 'SURRY COMMUNITY COLLEGE', 'SUSSEX COUNTY COMMUNITY COLLEGE', 'SW BAPTIST UNIVERSITY-SPRINGFIELD', 'SWARTHMORE COLLEGE', 'SWEET BRIAR COLLEGE', 'SYRACUSE UNIVERSITY', 'TABOR COLLEGE', 'TACOMA COMMUNITY COLLEGE', 'TAFT COLLEGE', 'TALLAHASSEE COMMUNITY COLLEGE', 'TARKIO COLLEGE', 'TARLETON STATE UNIVERSITY', 'TARRANT COUNTY CHRISTAN ACAD', 'TARRANT COUNTY COLLEGE DISTRICT (PRIMARY)', 'TAYLOR UNIVERSITY', 'TEACHERS COLLEGE SAN JOAQUIN', 'TECHNICAL CAREER INSTITUTES', 'TECHNICAL COLLEGE LOWCOUNTRY', 'TEMPLE COLLEGE', 'TEMPLE UNIVERSITY (PRIMARY)', 'TENNESSEE STATE UNIVERSITY', 'TENNESSEE TECHNOLOGICAL UNIVERSITY', 'TENNESSEE TEMPLE UNIVERSITY', 'TENNESSEE WESLEYAN COLLEGE', 'TERRA STATE COMMUNITY COLLEGE', 'TEXARKANA COLLEGE', 'TEXAS A&M INTERNATIONAL UNIVERSITY', 'TEXAS A&M UNIVERSITY', 'TEXAS A&M UNIVERSITY - CENTRAL TEXAS', 'TEXAS A&M UNIVERSITY - COMMERCE', 'TEXAS A&M UNIVERSITY - CORPUS CHRISTI', 'TEXAS A&M UNIVERSITY - GALVESTON', 'TEXAS A&M UNIVERSITY - KINGSVILLE', 'TEXAS A&M UNIVERSITY - SAN ANTONIO', 'TEXAS A&M UNIVERSITY - TEXARKANA', 'TEXAS CHRISTIAN UNIVERSITY', 'TEXAS LUTHERAN UNIVERSITY', 'TEXAS SOUTHERN UNIVERSITY', 'TEXAS SOUTHMOST COLLEGE', 'TEXAS STATE TECHNICAL COLLEGE (PRIMARY)', 'TEXAS STATE UNIVERSITY - SAN MARCOS', 'TEXAS TECH UNIVERSITY', 'TEXAS WESLEYAN UNIVERSITY', "TEXAS WOMAN'S UNIVERSITY", 'THADDEUS STEVENS COLL TECHNOLOGY', 'THIEL COLLEGE', 'THOMAS AQUINAS COLLEGE', 'THOMAS COLLEGE', 'THOMAS EDISON STATE UNIVERSITY', 'THOMAS JEFFERSON UNIVERSITY', 'THOMAS MORE COLL', 'THOMAS NELSON COMMUNITY COLLEGE', 'THOMAS UNIVERSITY', 'THREE RIVERS COLLEGE', 'THREE RIVERS COLLEGE (MO)', 'THREE RIVERS COMMUNITY COLLEGE (CT)', 'TIDEWATER COMMUNITY COLLEGE', 'TIFFIN UNIVERSITY', 'TOCCOA FALLS COLLEGE', 'TOMPKINS-CORTLAND COMMUNITY COLLEGE', 'TOUGALOO COLLEGE', 'TOURO COLLEGE', 'TOURO UNIVERSITY NEVADA', 'TOURO UNIVERSITY WORLDWIDE', 'TOWSON UNIVERSITY', 'TRANSYLVANIA UNIVERSITY', 'TRENHOLM STATE COMMUNITY COLLEGE (PRIMARY)', 'TREVECCA NAZARENE UNIVERSITY', 'TRI-COUNTY COMMUNITY COLLEGE', 'TRI-COUNTY TECHNICAL COLLEGE', 'TRIBHUVAN UNIVERSITY', 'TRIDENT AT AMERICAN INTERCONTINENTAL UNIVERSITY', 'TRIDENT TECHNICAL COLLEGE', 'TRINE UNIVERSITY', 'TRINIDAD ST JR COLL', 'TRINITY BAPTIST COLLEGE', 'TRINITY CHRISTIAN COLLEGE', 'TRINITY COLLEGE HARTFORD', 'TRINITY INTERNATIONAL UNIVERSITY (PRIMARY)', 'TRINITY UNIVERSITY', 'TRINITY VALLEY COMMUNITY COLLEGE (PRIMARY)', 'TRINITY WASHINGTON UNIVERSITY', 'TRINITY WESTERN UNIVERSITY', 'TRITON COLLEGE', 'TROCAIRE COLLEGE', 'TROY UNIVERSITY (PRIMARY)', 'TRUCKEE MEADOWS COMMUNITY COLLEGE', 'TRUETT MCCONNELL UNIVERSITY', 'TRUMAN STATE UNIVERSITY', 'TRUSTEES DARTMOUTH COLLEGE', 'TUFTS UNIVERSITY', 'TULANE UNIVERSITY', 'TULSA COMMUNITY COLLEGE', 'TUNXIS COMMUNITY COLLEGE', 'TURABO UNIVERSITY', 'TUSCULUM UNIVERSITY', 'TUSKEGEE UNIVERSITY', 'TYLER JUNIOR COLLEGE', 'UCSI UNIVERSITY', 'ULSTER COUNTY COMMUNITY COLLEGE (SUNY)', 'UMPQUA COMMUNITY COLLEGE', 'UNIFORMED SERVICES UNIVERSITY HEALTH SCIENCES', 'UNION COLLEGE (KY)', 'UNION COLLEGE (NE)', 'UNION COUNTY COLLEGE (NJ)', 'UNION INSTITUTE UNIVERSITY', 'UNION UNIVERSITY (PRIMARY)', 'UNITED STATES MILITARY ACADEMY', 'UNITED STATES NAVAL ACADEMY', 'UNITED STATES SPORTS ACADEMY', 'UNITED STATES UNIVERSITY', 'UNITY COLLEGE', 'UNITY ENVIRONMENTAL UNIVERSITY', 'UNIV ARKANSAS CC-BATESVILLE', 'UNIV ARKANSAS GRANTHAM', 'UNIV CHICAGO', 'UNIV DISTRICT COLUMBIA', 'UNIV DUBUQUE', 'UNIV GUAM', 'UNIV HARTFORD', 'UNIV HOUSTON VICTORA', 'UNIV LA VERNE', 'UNIV MAINE AUGUSTA', 'UNIV MAINE FARMINGTN', 'UNIV MAINE FT KENT', 'UNIV MARYLAND MUNICH', 'UNIV MICH DEARBORN', 'UNIV MICHIGAN ANN AR', 'UNIV MINNESOTA ROCHESTER', 'UNIV MONTANA WESTERN', 'UNIV NORTH CAROLINA PEMBROKE', 'UNIV PITTSBRGH BRADF', 'UNIV PITTSBURGH AT JOHNSTOWN', 'UNIV PITTSBURGH AT TITUSVILLE', 'UNIV PRINCE EDWARD ISLAND', 'UNIV RICHMOND', 'UNIV SCRANTON', 'UNIV SOUTH', 'UNIV SOUTH CAROLINA BEAUFORT', 'UNIV SOUTH CAROLINA-AIKEN', 'UNIV SOUTH CAROLINA-UPSTATE', 'UNIV TENNESSEE', 'UNIV TENNESSEE SOUTHERN', "UNIV VIRGINIA'S COLLEGE AT WISE", 'UNIV WISC PLATTEVLLE', 'UNIV WISCONSIN - RIVER FALLS', 'UNIV. MISSOURI-COLUMBIA H. S.', 'UNIVERSIDAD ADVENTISTA DE LAS ANTILLAS', 'UNIVERSIDAD DEL SAGRADO CORAZON', 'UNIVERSIDAD SAN FRANCISCO DE QUITO', 'UNIVERSITA TELEMATICA E-CAMPUS', 'UNIVERSITY ADVANCING TECHNOLOGY', 'UNIVERSITY AKRON (PRIMARY)', 'UNIVERSITY ALABAMA', 'UNIVERSITY ALABAMA AT BIRMINGHAM', 'UNIVERSITY ALABAMA IN HUNTSVILLE', 'UNIVERSITY ALASKA ANCHORAGE (PRIMARY)', 'UNIVERSITY ALASKA FAIRBANKS (PRIMARY)', 'UNIVERSITY ALASKA SOUTHEAST (PRIMARY)', 'UNIVERSITY ANTELOPE VALLEY', 'UNIVERSITY ARIZONA', 'UNIVERSITY ARIZONA GLOBAL CAMPUS (FORMERLY ASHFORD U)', 'UNIVERSITY ARKANSAS', 'UNIVERSITY ARKANSAS AT LITTLE ROCK', 'UNIVERSITY ARKANSAS AT MONTICELLO', 'UNIVERSITY ARKANSAS AT PINE BLUFF', 'UNIVERSITY ARKANSAS COMMUNITY COLLEGE - HOPE', 'UNIVERSITY ARKANSAS COMMUNITY COLLEGE AT MORRILTON', 'UNIVERSITY ARKANSAS FOR MEDICAL SCIENCES', 'UNIVERSITY ARKANSAS FORT SMITH', 'UNIVERSITY ARTS', 'UNIVERSITY AT BUFFALO, STATE UNIVERSITY NEW YORK', 'UNIVERSITY BALTIMORE', 'UNIVERSITY BRIDGEPORT', 'UNIVERSITY BRITISH COLUMBIA', 'UNIVERSITY CALIFORNIA - BERKELEY', 'UNIVERSITY CALIFORNIA - DAVIS', 'UNIVERSITY CALIFORNIA - IRVINE', 'UNIVERSITY CALIFORNIA - LOS ANGELES', 'UNIVERSITY CALIFORNIA - MERCED', 'UNIVERSITY CALIFORNIA - RIVERSIDE', 'UNIVERSITY CALIFORNIA - SAN DIEGO', 'UNIVERSITY CALIFORNIA - SAN FRANCISCO', 'UNIVERSITY CALIFORNIA - SANTA BARBARA', 'UNIVERSITY CALIFORNIA - SANTA CRUZ', 'UNIVERSITY CENTRAL ARKANSAS', 'UNIVERSITY CENTRAL FLORIDA', 'UNIVERSITY CENTRAL MISSOURI', 'UNIVERSITY CENTRAL OKLAHOMA', 'UNIVERSITY CHARLESTON (PRIMARY)', 'UNIVERSITY CINCINNATI', 'UNIVERSITY COLORADO - COLORADO SPRINGS', 'UNIVERSITY COLORADO BOULDER', 'UNIVERSITY COLORADO DENVER', 'UNIVERSITY CONNECTICUT', 'UNIVERSITY CUMBERLANDS', 'UNIVERSITY DALLAS', 'UNIVERSITY DAYTON', 'UNIVERSITY DELAWARE', 'UNIVERSITY DENVER', 'UNIVERSITY DETROIT MERCY', 'UNIVERSITY EDINBURGH', 'UNIVERSITY EVANSVILLE', 'UNIVERSITY FINDLAY', 'UNIVERSITY FLORIDA', 'UNIVERSITY GEORGIA ATHENS', 'UNIVERSITY GRAZ', 'UNIVERSITY HAWAII - MAUI COLLEGE', 'UNIVERSITY HAWAII AT HILO', 'UNIVERSITY HAWAII AT MANOA', "UNIVERSITY HAWAII-WEST O'AHU", 'UNIVERSITY HEALTH SCIENCES PHARMACY IN ST. LOUIS', 'UNIVERSITY HOUSTON (MAIN CAMPUS)', 'UNIVERSITY HOUSTON - CLEAR LAKE', 'UNIVERSITY HOUSTON-DWNTN C', 'UNIVERSITY IDAHO', 'UNIVERSITY ILLINOIS AT CHICAGO', 'UNIVERSITY ILLINOIS AT SPRINGFIELD', 'UNIVERSITY ILLINOIS AT URBANA-CHAMPAIGN', 'UNIVERSITY INCARNATE WORD', 'UNIVERSITY INDIANAPOLIS', 'UNIVERSITY IOWA', 'UNIVERSITY JAMESTOWN', 'UNIVERSITY KANSAS', 'UNIVERSITY KENTUCKY', 'UNIVERSITY LOUISIANA AT LAFAYETTE', 'UNIVERSITY LOUISIANA AT MONROE', 'UNIVERSITY LOUISVILLE', 'UNIVERSITY LYNCHBURG', 'UNIVERSITY MAINE', 'UNIVERSITY MAINE AT PREQUE ISLE', 'UNIVERSITY MARY', 'UNIVERSITY MARY HARDIN - BAYLOR', 'UNIVERSITY MARY WASHINGTON', 'UNIVERSITY MARYLAND BALTIMORE', 'UNIVERSITY MARYLAND BALTIMORE COUNTY', 'UNIVERSITY MARYLAND COLLEGE PARK', 'UNIVERSITY MARYLAND EASTERN SHORE', 'UNIVERSITY MARYLAND GLOBAL CAMPUS', 'UNIVERSITY MASSACHUSETTS', 'UNIVERSITY MASSACHUSETTS - BOSTON', 'UNIVERSITY MASSACHUSETTS - DARTMOUTH', 'UNIVERSITY MASSACHUSETTS AMHERST', 'UNIVERSITY MASSACHUSETTS GLOBAL', 'UNIVERSITY MEDICINE DENTISTRY NEW JERSEY', 'UNIVERSITY MEMPHIS', 'UNIVERSITY MIAMI', 'UNIVERSITY MICHIGAN - FLINT', 'UNIVERSITY MINNESOTA - CROOKSTON', 'UNIVERSITY MINNESOTA - DULUTH', 'UNIVERSITY MINNESOTA - TWIN CITIES', 'UNIVERSITY MISSISSIPPI', 'UNIVERSITY MISSISSIPPI MEDICAL CENTER', 'UNIVERSITY MISSOURI - COLUMBIA', 'UNIVERSITY MISSOURI - SAINT LOUIS', 'UNIVERSITY MISSOURI KANSAS CITY', 'UNIVERSITY MISSOURI-ST LOUIS', 'UNIVERSITY MOBILE', 'UNIVERSITY MONTANA', 'UNIVERSITY MONTEVALLO', 'UNIVERSITY MOUNT OLIVE (PRIMARY)', 'UNIVERSITY NEBRASKA', 'UNIVERSITY NEBRASKA AT KEARNEY', 'UNIVERSITY NEBRASKA AT OMAHA', 'UNIVERSITY NEBRASKA MEDICAL CENTER', 'UNIVERSITY NEVADA LAS VEGAS', 'UNIVERSITY NEVADA RENO', 'UNIVERSITY NEW ENGLAND', 'UNIVERSITY NEW HAMPSHIRE', 'UNIVERSITY NEW HAVEN', 'UNIVERSITY NEW MEXICO', 'UNIVERSITY NEW ORLEANS', 'UNIVERSITY NIGERIA', 'UNIVERSITY NORTH ALABAMA', 'UNIVERSITY NORTH CAROLINA AT ASHEVILLE', 'UNIVERSITY NORTH CAROLINA AT CHAPEL HILL', 'UNIVERSITY NORTH CAROLINA AT CHARLOTTE', 'UNIVERSITY NORTH CAROLINA AT GREENSBORO', 'UNIVERSITY NORTH CAROLINA AT WILMINGTON', 'UNIVERSITY NORTH DAKOTA', 'UNIVERSITY NORTH FLORIDA', 'UNIVERSITY NORTH GEORGIA', 'UNIVERSITY NORTH TEXAS', 'UNIVERSITY NORTH TEXAS DALLAS', 'UNIVERSITY NORTHERN COLORADO', 'UNIVERSITY NORTHERN IOWA', 'UNIVERSITY NORTHWESTERN (MN)', 'UNIVERSITY NORTHWESTERN OHIO', 'UNIVERSITY NOTRE DAME', 'UNIVERSITY OKLAHOMA', 'UNIVERSITY OKLAHOMA - HEALTH SCIENCES', 'UNIVERSITY OREGON', 'UNIVERSITY OZARKS (AR)', 'UNIVERSITY PACIFIC', 'UNIVERSITY PENNSYLVANIA', 'UNIVERSITY PHOENIX', 'UNIVERSITY PIKEVILLE', 'UNIVERSITY PITTSBURGH', 'UNIVERSITY PROVIDENCE', 'UNIVERSITY PUERTO RICO', 'UNIVERSITY PUGET SOUND', 'UNIVERSITY READING', 'UNIVERSITY REDLANDS', 'UNIVERSITY RHODE ISLAND', 'UNIVERSITY RIO GRANDE', 'UNIVERSITY ROCHESTER', 'UNIVERSITY SAINT FRANCIS (IN)', 'UNIVERSITY SAINT JOSEPH', 'UNIVERSITY SAINT MARY', 'UNIVERSITY SAINT THOMAS', 'UNIVERSITY SAN AGUSTIN', 'UNIVERSITY SAN DIEGO', 'UNIVERSITY SAN FRANCISCO', 'UNIVERSITY SANTO TOMAS', 'UNIVERSITY SASKATCHEWAN', 'UNIVERSITY SCIENCE ARTS OKLAHOMA', 'UNIVERSITY SCIENCES PHILADELPHIA', 'UNIVERSITY SIOUX FALLS', 'UNIVERSITY SOUTH ALABAMA', 'UNIVERSITY SOUTH CAROLINA', 'UNIVERSITY SOUTH CAROLINA SUMTER', 'UNIVERSITY SOUTH DAKOTA', 'UNIVERSITY SOUTH FLORIDA', 'UNIVERSITY SOUTHERN CALIFORNIA', 'UNIVERSITY SOUTHERN INDIANA', 'UNIVERSITY SOUTHERN MAINE', 'UNIVERSITY SOUTHERN MISSISSIPPI (PRIMARY)', 'UNIVERSITY ST FRANCIS (IL)', 'UNIVERSITY ST. AUGUSTINE FOR HEALTH SCIENCES', 'UNIVERSITY ST. THOMAS (MN)', 'UNIVERSITY TAMPA', 'UNIVERSITY TENNESSEE', 'UNIVERSITY TENNESSEE AT CHATTANOOGA', 'UNIVERSITY TENNESSEE AT MARTIN', 'UNIVERSITY TENNESSEE HEALTH SCIENCE CENTER', 'UNIVERSITY TEXAS - DALLAS', 'UNIVERSITY TEXAS - RIO GRANDE VALLEY', 'UNIVERSITY TEXAS AT ARLINGTON', 'UNIVERSITY TEXAS AT AUSTIN', 'UNIVERSITY TEXAS AT SAN ANTONIO', 'UNIVERSITY TEXAS AT TYLER', 'UNIVERSITY TEXAS EL PASO', 'UNIVERSITY TEXAS HEALTH SCIENCE CENTER AT HOUSTON', 'UNIVERSITY TEXAS HEALTH SCIENCE CENTER AT SAN ANTONIO', 'UNIVERSITY TEXAS MEDICAL BRANCH', 'UNIVERSITY TEXAS PERMIAN BASIN', 'UNIVERSITY TOLEDO', 'UNIVERSITY TULSA', 'UNIVERSITY UTAH', 'UNIVERSITY VALLEY FORGE', 'UNIVERSITY VERMONT', 'UNIVERSITY VICTORIA', 'UNIVERSITY VIRGIN ISLANDS', 'UNIVERSITY VIRGINIA', 'UNIVERSITY WASHINGTON', 'UNIVERSITY WASHINGTON TACOMA', 'UNIVERSITY WEST ALABAMA', 'UNIVERSITY WEST FLORIDA', 'UNIVERSITY WEST GEORGIA', 'UNIVERSITY WEST INDIES', 'UNIVERSITY WISCONSIN - EAU CLAIRE', 'UNIVERSITY WISCONSIN - LA CROSSE', 'UNIVERSITY WISCONSIN - MADISON', 'UNIVERSITY WISCONSIN - MILWAUKEE', 'UNIVERSITY WISCONSIN - PARKSIDE', 'UNIVERSITY WISCONSIN - PLATTEVILLE', 'UNIVERSITY WISCONSIN - STEVENS POINT', 'UNIVERSITY WISCONSIN - STOUT', 'UNIVERSITY WISCONSIN - SUPERIOR', 'UNIVERSITY WISCONSIN - WHITEWATER', 'UNIVERSITY WISCONSIN GREEN BAY', 'UNIVERSITY WISCONSIN OSHKOSH', 'UNIVERSITY WYOMING', 'UNKNOWN INSTITUTION', 'UPPER IOWA UNIVERSITY', 'URBAN COLLEGE BOSTON', 'URSHAN COLLEGE', 'URSINUS COLLEGE', 'URSULINE COLLEGE', 'US AIR FORCE ACADEMY', 'US COAST GUARD ACAD', 'UTAH STATE UNIVERSITY', 'UTAH TECH UNIVERSITY', 'UTAH VALLEY UNIVERSITY', 'UTICA COLLEGE', 'VALDOSTA STATE UNIVERSITY', 'VALENCIA COLLEGE', 'VALLEY CITY STATE UNIVERSITY', 'VALLEY FORGE MILITARY ACADEMY COLLEGE', 'VALPARAISO UNIVERSITY', 'VANCE-GRANVILLE COMMUNITY COLLEGE', 'VANDERBILT UNIVERSITY', 'VANGUARD UNIV SOUTHERN CA', 'VASSAR COLL', 'VAUGHN COLL AERONAUTICS/TECH', 'VENTURA COLLEGE', 'VERMILION COMMUNITY COLLEGE', 'VERMONT TECHNICAL COLLEGE', 'VERNON COLLEGE', 'VICTOR VALLEY COLLEGE', 'VICTORIA COLLEGE', 'VILLA MARIA COLL NY', 'VILLANOVA UNIVERSITY', 'VINCENNES UNIVERSITY', 'VIRGINIA COMMONWEALTH UNIVERSITY', 'VIRGINIA HIGHLANDS COMMUNITY COLLEGE', 'VIRGINIA INTERMONT COLLEGE', 'VIRGINIA MILTRY INST', 'VIRGINIA PENINSULA COMMUNITY COLLEGE', 'VIRGINIA POLYTECHNIC INSTITUTE STATE UNIVERSITY', 'VIRGINIA STATE UNIV', 'VIRGINIA UNION UNIV', 'VIRGINIA WESLEYAN UNIVERSITY', 'VIRGINIA WESTERN COMMUNITY COLLEGE', 'VITERBO UNIVERSITY', 'VOLUNTEER STATE COMMUNITY COLLEGE (PRIMARY)', 'VOORHEES COLLEGE', 'WADE COLLEGE', 'WAGNER COLLEGE', 'WAKE FOREST UNIVERSITY', 'WAKE TECHNICAL COMMUNITY COLLEGE', 'WALDEN UNIVERSITY', 'WALDORF UNIVERSITY', 'WALLA WALLA C C', 'WALLA WALLA COLL', 'WALLACE COMMUNITY COLLEGE', 'WALLACE STATE COMMUNITY COLLEGE', 'WALTERS STATE COMMUNITY COLLEGE', 'WARNER PACIFIC COLL', 'WARREN COUNTY COMMUNITY COLLEGE', 'WARREN WILSON COLLEGE', 'WARTBURG COLLEGE', 'WASHBURN UNIVERSITY TOPEKA', 'WASHINGTON ADVENTIST UNIVERSITY', 'WASHINGTON COLLEGE', 'WASHINGTON JEFFERSON COLLEGE', 'WASHINGTON STATE COMMUNITY COLLEGE (OH)', 'WASHINGTON STATE UNIVERSITY', 'WASHINGTON UNIVERSITY IN SAINT LOUIS', 'WASHINGTON UNIVERSITY IN ST. LOUIS', 'WASHTENAW COMMUNITY COLLEGE', 'WATTERSON COLLEGE', 'WAUBONSEE COMMUNITY COLLEGE', 'WAUKESHA COUNTY TECHNICAL COLLEGE', 'WAYLAND BAPTIST UNIVERSITY (PRIMARY)', 'WAYNE COMMUNITY COLLEGE', 'WAYNE COUNTY COMMUNITY COLLEGE', 'WAYNE STATE COLLEGE', 'WAYNE STATE UNIVERSITY', 'WAYNESBURG COLL', 'WEATHERFORD COLLEGE', 'WEBER STATE UNIVERSITY', 'WELCH COLLEGE', 'WELLS COLLEGE', 'WENATCHEE VALLEY COLLEGE', 'WENTWORTH INST TECH', 'WENTWORTH MILITARY ACADEMY', 'WESLEY COLLEGE (DE)', 'WESLEYAN COLL', 'WESLEYAN UNIV', 'WEST CHESTER UNIVERSITY PENNSYLVANIA', 'WEST COAST UNIVERSITY (PRIMARY)', 'WEST GEORGIA TECHNICAL COLLEGE', 'WEST HILLS COMMUNITY COLLEGE DISTRICT (PRIMARY)', 'WEST LIBERTY UNIVERSITY', 'WEST LOS ANGELES COLLEGE', 'WEST SHORE COMMUNITY COLLEGE', 'WEST TEXAS A&M UNIVERSITY', 'WEST VALLEY COLLEGE', 'WEST VIRGINIA NORTHERN COMMUNITY COLLEGE (PRIMARY)', 'WEST VIRGINIA STATE UNIVERSITY', 'WEST VIRGINIA UNIVERSITY', 'WEST VIRGINIA UNIVERSITY AT PARKERSBURG', 'WEST VIRGINIA UNIVERSITY INSTITUTE TECHNOLOGY', 'WEST VIRGINIA WESLEYAN COLLEGE', 'WESTCHESTER COMMUNITY COLLEGE', 'WESTERN CAROLINA UNIVERSITY', 'WESTERN COLORADO UNIVERSITY', 'WESTERN CONNECTICUT STATE UNIVERSITY', 'WESTERN DAKOTA TECHNICAL INSTITUTE', 'WESTERN GOVERNORS UNIVERSITY', 'WESTERN ILLINOIS UNIVERSITY', 'WESTERN INTERNATIONAL UNIVERSITY', 'WESTERN IOWA TECH COMMUNITY COLLEGE', 'WESTERN KENTUCKY UNIVERSITY', 'WESTERN MICHIGAN UNIVERSITY', 'WESTERN NEBRASKA COMMUNITY COLLEGE', 'WESTERN NEVADA COMMUNITY COLLEGE', 'WESTERN NEW ENGLAND UNIVERSITY', 'WESTERN NEW MEXICO UNIVERSITY', 'WESTERN OKLAHOMA STATE COLLEGE', 'WESTERN OREGON UNIVERSITY', 'WESTERN TECHNICAL COLLEGE', 'WESTERN TEXAS COLL', 'WESTERN UNIVERSITY HEALTH SCIENCES', 'WESTERN WASHINGTON UNIVERSITY', 'WESTERN WYOMING COMMUNITY COLLEGE', 'WESTFIELD STATE UNIVERSITY', 'WESTMINSTER COLLEGE', 'WESTMINSTER COLLEGE (UT)', 'WESTMONT COLL', 'WESTMORELAND COUNTY COMMUNITY COLLEGE', 'WESTRN PIEDMONT C C', 'WHARTON COUNTY JUNIOR COLLEGE', 'WHATCOM COMMUNITY COLLEGE', 'WHEATON COLLEGE (IL)', 'WHEELING JESUIT UNIVERSITY', 'WHEELOCK COLLEGE', 'WHITE MOUNTAINS COMMUNITY COLLEGE', 'WHITTIER COLLEGE', 'WHITWORTH UNIVERSITY', 'WICHITA STATE UNIVERSITY', 'WICHITA STATE UNIVERSITY CAMPUS APPLIED SCIENCES/TECH', 'WIDENER UNIVERSITY', 'WILBERFORCE UNIVERSITY', 'WILEY COLLEGE', 'WILKES COMMUNITY COLLEGE', 'WILKES UNIVERSITY', 'WILLAMETTE UNIV', 'WILLIAM CAREY UNIVERSITY', 'WILLIAM JESSUP UNIVERSITY', 'WILLIAM JEWELL COLLEGE', 'WILLIAM PATERSON UNIVERSITY NEW JERSEY', 'WILLIAM PEACE UNIVERSITY', 'WILLIAM PENN UNIVERSITY', 'WILLIAM TYNDALE COLLEGE', 'WILLIAM WOODS UNIVERSITY', 'WILLIAMS BAPTIST UNIVERSITY', 'WILLIAMSBURG TECHNICAL COLLEGE', 'WILLISTON STATE COLLEGE', 'WILMINGTON COLLEGE OHIO', 'WILMINGTON UNIVERSITY', 'WILMINGTON UNIVERSITY - DOVER AIR FORCE BASE', 'WILSON COLLEGE', 'WILSON COMMUNITY COLLEGE', 'WINDWARD COMMUNITY COLLEGE', 'WINGATE UNIVERSITY', 'WINONA STATE UNIVERSITY', 'WINSTON-SALEM STATE UNIVERSITY', 'WINTHROP UNIVERSITY', 'WIREGRASS GEORGIA TECHNICAL COLLEGE (PRIMARY)', 'WITTENBERG UNIV', 'WOFFORD COLLEGE', 'WOODBURY UNIV', 'WOR-WIC COMMUNITY COLLEGE', 'WORCESTER POLYTECHNIC INSTITUTE', 'WORCESTER ST COLL', 'WRIGHT STATE UNIVERSITY (PRIMARY)', 'WS-337U', 'WYTHEVILLE COMMUNITY COLLEGE', 'XAVIER UNIVERSITY', 'XAVIER UNIVERSITY (OH)', 'XAVIER UNIVERSITY LOUISIANA', 'XRAY-211', 'YAKIMA VALLEY COLL', 'YALE UNIVERSITY', 'YAVAPAI COLL', 'YESHIVA UNIV', 'YORK COLLEGE (CUNY)', 'YORK COLLEGE (NE)', 'YORK COLLEGE PENNSYLVANIA', 'YORK COUNTY COMM COLL', 'YORK TECHNICAL COLLEGE', 'YORK UNIV', 'YOUNG HARRIS COLLEGE', 'YOUNGSTOWN STATE UNIVERSITY', 'YUBA COLLEGE', 'ZANE STATE COLLEGE', 'ZEN-101']
len(schools)

2686

In [5]:
t = timer("Lookup")
d_results = {}
for i, school_name in enumerate(schools):

    if i > 0 and i % 250 == 0:
        print (f"{i} - {len(schools)}")
    
    d_results[school_name] = lookup(school_name)

len(d_results)
t()

250 - 2686
500 - 2686
750 - 2686
1000 - 2686
1250 - 2686
1500 - 2686
1750 - 2686
2000 - 2686
2250 - 2686
2500 - 2686
⏱️  Lookup completed in 837.04s (13.95 min)


In [8]:
def write_json_to_disk(data: list[str], output_path: str) -> None:
    path = Path(output_path)
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"💾 Wrote {len(data)} unique schools → {output_path}")

write_json_to_disk(d_results, "/home/craigtrim/projects/colleage-courses.json")

💾 Wrote 2686 unique schools → /home/craigtrim/projects/colleage-courses.json


In [18]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import csv
from sentence_transformers import SentenceTransformer, util

# ---------------------------------------------------------------------
# Paths
# ---------------------------------------------------------------------
CSV_PATH = "/home/craigtrim/projects/alias-label-retriever/data/temp/crosswalk-candidates.csv"
OUTPUT_PATH = "/home/craigtrim/projects/alias-label-retriever/data/temp/crosswalk-candidates-with-synonyms.csv"
MODEL_PATH = "/home/craigtrim/projects/alias-label-retriever/models/trained/alias_label_e5"

# ---------------------------------------------------------------------
# Load data and model
# ---------------------------------------------------------------------
rows = []
with open(CSV_PATH, "r", encoding="utf-8") as f:
    reader = csv.DictReader(f, delimiter="\t")
    for row in reader:
        rows.append(row)

model = SentenceTransformer(MODEL_PATH)

# Collect all unique potential names
all_names = sorted({
    name.strip()
    for row in rows
    for name in (row.get("colleague", ""), row.get("label", ""))
    if name and name.strip()
})

# Encode once
embeddings = model.encode(all_names, convert_to_tensor=True, normalize_embeddings=True)

# ---------------------------------------------------------------------
# Function to generate synonyms
# ---------------------------------------------------------------------
def get_synonyms(name: str, exclude: set[str], top_k: int = 10, min_conf: float = 0.93) -> list[str]:
    """Return up to top_k unique high-confidence synonyms not in exclude."""
    query_emb = model.encode(name, convert_to_tensor=True, normalize_embeddings=True)
    cos_scores = util.cos_sim(query_emb, embeddings)[0]
    top_results = cos_scores.topk(top_k + 1)  # +1 to skip self-match

    synonyms = []
    for idx, score in zip(top_results.indices, top_results.values):
        candidate = all_names[idx].strip()
        conf = float(score)
        if candidate.lower() in {e.lower() for e in exclude}:
            continue  # skip duplicates or original names
        if conf >= min_conf:
            synonyms.append(candidate)

    return sorted(set(synonyms))[:top_k]


# ---------------------------------------------------------------------
# Generate synonyms
# ---------------------------------------------------------------------
for row in rows:
    names_to_expand = [row.get("colleague", ""), row.get("label", "")]
    exclude_set = {n for n in names_to_expand if n}
    all_syns = set()
    for name in names_to_expand:
        if name and name.strip():
            all_syns.update(get_synonyms(name, exclude_set))
    row["synonyms"] = "#".join(sorted(all_syns))

# ---------------------------------------------------------------------
# Write output (comma-delimited, quoted)
# ---------------------------------------------------------------------
fieldnames = list(rows[0].keys())
with open(OUTPUT_PATH, "w", encoding="utf-8", newline="") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=fieldnames,
        delimiter=",",
        quotechar='"',
        quoting=csv.QUOTE_ALL
    )
    writer.writeheader()
    writer.writerows(rows)

print(f"✅ Wrote CSV with synonyms (Excel-safe quoting) → {OUTPUT_PATH}")

✅ Wrote CSV with synonyms (Excel-safe quoting) → /home/craigtrim/projects/alias-label-retriever/data/temp/crosswalk-candidates-with-synonyms.csv
